In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import gc
from tqdm.auto import tqdm
import os
import optuna
tqdm.pandas()

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Libraries imported successfully.")

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


Libraries imported successfully.


In [2]:
USE_GPU = True  

gpu_config = {
    'xgboost_gpu': USE_GPU,
    'lightgbm_gpu': USE_GPU,
    'catboost_gpu': USE_GPU
}

In [3]:
class Config:
    DATA_PATH = '/kaggle/input/anava-dataset'
    
    N_FOLDS = 5
    RANDOM_STATE = 42
    TARGET_COL = 'Trip_Label'
    ID_COL = 'Trip_ID'
    
    USE_TEMPORAL = False
    USE_DISTANCE = False
    USE_SENSOR_AGG = False
    USE_ECONOMIC = False
    USE_INTERACTION = False
    
    @staticmethod
    def get_catboost_params(use_gpu=False):
        params = {
            'iterations': 1000,
            'learning_rate': 0.05,
            'depth': 6,
            'loss_function': 'MultiClass',
            'eval_metric': 'TotalF1:average=Macro',
            'auto_class_weights': 'Balanced',
            'random_seed': 42,
            'verbose': 100,
            'early_stopping_rounds': 50
        }
        if use_gpu:
            params['task_type'] = 'GPU'
            params['devices'] = '0'
            print("  CatBoost: GPU mode activated")
        else:
            params['task_type'] = 'CPU'
            print("  CatBoost: CPU mode")
        return params
    
    @staticmethod
    def get_lightgbm_params(use_gpu=False):
        params = {
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'random_state': 42,
            'verbose': 1,
            'min_data_in_leaf': 200,
            'min_sum_hessian_in_leaf': 1e-2,
            'gpu_platform_id': 0,
            'gpu_device_id': 0,
            'max_bin': 255,
            'force_col_wise': True
        }
        if use_gpu:
            params['device'] = 'gpu'
            print("  LightGBM: GPU mode activated")
        else:
            params['device'] = 'cpu'
            print("  LightGBM: CPU mode")
        return params
    
    @staticmethod
    def get_xgboost_params(use_gpu=False):
        params = {
            'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'max_depth': 6,
            'learning_rate': 0.05,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'verbosity': 1
        }
        if use_gpu:
            params['device'] = 'cuda'
            print("  XGBoost: GPU mode activated")
        else:
            params['device'] = 'cpu'
            print("  XGBoost: CPU mode")
        return params

config = Config()
print("\nConfiguration loaded successfully.")


Configuration loaded successfully.


In [4]:
def load_data():
    print("Loading data...")

    files = ['train.csv', 'test.csv', 'sample_submission.csv']
    data = {}

    for file in tqdm(files, desc="Loading files"):
        file_path = os.path.join(config.DATA_PATH, file)
        data[file.replace('.csv', '')] = pd.read_csv(file_path)

    train = data['train']
    test = data['test']
    sample_submission = data['sample_submission']

    print(f"Train shape: {train.shape}")
    print(f"Test shape: {test.shape}")
    print(f"Sample submission shape: {sample_submission.shape}")

    if config.TARGET_COL in train.columns:
        print("\nTarget distribution:")
        print(train[config.TARGET_COL].value_counts())

    return train, test, sample_submission

train, test, sample_submission = load_data()

Loading data...


Loading files:   0%|          | 0/3 [00:00<?, ?it/s]

Train shape: (8000000, 25)
Test shape: (4000000, 24)
Sample submission shape: (4000000, 2)

Target distribution:
Trip_Label
Perfect_Trip         4397607
Safety_Violation     1601595
Navigation_Issue      801790
Service_Complaint     798695
Fraud_Indication      400313
Name: count, dtype: int64


In [5]:
def optimize_memory(df):
    print(f"Optimizing memory for dataframe with {len(df.columns)} columns...")
    for col in tqdm(df.columns, desc="Optimizing columns"):
        col_type = df[col].dtype
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    return df

print("Optimizing memory...")
train = optimize_memory(train)
test = optimize_memory(test)
print("Memory optimization completed.")

Optimizing memory...
Optimizing memory for dataframe with 25 columns...


Optimizing columns:   0%|          | 0/25 [00:00<?, ?it/s]

Optimizing memory for dataframe with 24 columns...


Optimizing columns:   0%|          | 0/24 [00:00<?, ?it/s]

Memory optimization completed.


In [6]:
# def haversine_distance(lat1, lon1, lat2, lon2):
#     R = 6371
#     lat1_rad = np.radians(lat1)
#     lat2_rad = np.radians(lat2)
#     delta_lat = np.radians(lat2 - lat1)
#     delta_lon = np.radians(lon2 - lon1)
    
#     a = np.sin(delta_lat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon/2)**2
#     c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
#     return R * c

# def bearing(lat1, lon1, lat2, lon2):
#     lat1, lat2 = np.radians(lat1), np.radians(lat2)
#     diff = np.radians(lon2 - lon1)
#     x = np.sin(diff) * np.cos(lat2)
#     y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(diff)
#     return np.degrees(np.arctan2(x, y))

# def geo_bin(lat, lon, precision=2):
#     return lat.round(precision).astype(str) + '_' + lon.round(precision).astype(str)
    
# def engineer_features(df, is_train=True):
#     print(f"Engineering features for {'train' if is_train else 'test'} set...")
#     df = df.copy()
    
#     if config.USE_TEMPORAL and 'Timestamp' in df.columns:
#         print("  - Creating temporal features...")
#         df['Timestamp_parsed'] = pd.to_datetime(df['Timestamp'], errors='coerce')
#         df['Hour'] = df['Timestamp_parsed'].dt.hour
#         df['DayOfWeek'] = df['Timestamp_parsed'].dt.dayofweek
#         df['Month'] = df['Timestamp_parsed'].dt.month
#         df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(float)
#         df['IsRushHour'] = ((df['Hour'] >= 7) & (df['Hour'] <= 9) | 
#                             (df['Hour'] >= 17) & (df['Hour'] <= 19)).astype(float)
#         df['IsLateNight'] = ((df['Hour'] >= 22) | (df['Hour'] <= 5)).astype(float)
#         df.drop('Timestamp_parsed', axis=1, inplace=True)
    
#     if config.USE_DISTANCE:
#         print("  - Creating distance features...")
#         if all(col in df.columns for col in ['Pickup_Lat', 'Pickup_Long', 'Dropoff_Lat', 'Dropoff_Long']):
#             df['Haversine_Distance'] = haversine_distance(
#                 df['Pickup_Lat'], df['Pickup_Long'],
#                 df['Dropoff_Lat'], df['Dropoff_Long']
#             )
#             df['Delta_Lat'] = df['Dropoff_Lat'] - df['Pickup_Lat']
#             df['Bearing'] = bearing(df.Pickup_Lat, df.Pickup_Long, df.Dropoff_Lat, df.Dropoff_Long)
#             df['pickup_grid'] = geo_bin(df['Pickup_Lat'], df['Pickup_Long'])
            
#             if 'Distance_KM' in df.columns:
#                 df['Distance_Ratio'] = df['Distance_KM'] / (df['Haversine_Distance'] + 1e-6)
#                 df['Distance_Difference'] = np.abs(df['Distance_KM'] - df['Haversine_Distance'])
#                 df['Is_Ultra_Short_Distance'] = (df['Distance_KM'] < 0.024).astype(int)
        
#         if 'Pickup_Zone' in df.columns and 'Dropoff_Zone' in df.columns:
#             df['Is_Same_Zone'] = (df['Pickup_Zone'] == df['Dropoff_Zone']).astype(np.int8)
            
#             pickup_freq = df['Pickup_Zone'].value_counts()
#             dropoff_freq = df['Dropoff_Zone'].value_counts()
            
#             df['pickup_zone_count'] = df['Pickup_Zone'].map(pickup_freq).fillna(0)
#             df['dropoff_zone_count'] = df['Dropoff_Zone'].map(dropoff_freq).fillna(0)
            
#     if config.USE_SENSOR_AGG:
#         print("  - Creating sensor aggregation features...")
#         if all(col in df.columns for col in ['Accel_X', 'Accel_Y', 'Accel_Z']):
#             df['Accel_Magnitude'] = np.sqrt(df['Accel_X']**2 + df['Accel_Y']**2 + df['Accel_Z']**2)
#             df['Accel_Max'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].max(axis=1)
#             df['Accel_Min'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].min(axis=1)
#             df['Accel_Std'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].std(axis=1)
#             df['Accel_Range'] = df['Accel_Max'] - df['Accel_Min']
#             df['Is_Accel_Magnitude_Outlier'] = (df['Accel_Magnitude'].abs() >= 10.80)
        
#         if 'Gyro_Z' in df.columns:
#             df['Gyro_Abs'] = np.abs(df['Gyro_Z'])
#             df['Is_Gyro_Z_Outlier'] = (df['Gyro_Z'].abs() >= 0.672).astype(int)
    
#     if config.USE_ECONOMIC:
#         print("  - Creating economic features...")
#         if 'Est_Price_IDR' in df.columns and 'Distance_KM' in df.columns:
#             df['Price_per_KM'] = df['Est_Price_IDR'] / (df['Distance_KM'] + 1e-6)

#         def encode_promo(x):
#             if pd.isna(x):
#                 return 0      # MISSING
#             if x == 'NO VOUCHER':
#                 return 1      # NO_VOUCHER
#             return 2          # HAS_VOUCHER
    
#         if 'Promo_Code' in df.columns:
#             df['Promo_Code_enc'] = df['Promo_Code'].apply(encode_promo).astype(np.int8)
#         if 'Surge_Multiplier' in df.columns:
#             surge_filled = df['Surge_Multiplier'].fillna(1.0)
    
#             df['Surge_Category'] = pd.cut(
#                 surge_filled,
#                 bins=[0, 1, 1.5, 2, 10],
#                 labels=[0, 1, 2, 3]
#             ).astype(np.int8)
    
#     if config.USE_INTERACTION:
#         print("  - Creating interaction features...")
#         if 'Surge_Multiplier' in df.columns and 'Hour' in df.columns:
#             df['Surge_Hour_Interaction'] = df['Surge_Multiplier'] * df['Hour']
        
#         if 'Distance_KM' in df.columns and 'Traffic' in df.columns:
#             traffic_map = {'Light': 1, 'Moderate': 2, 'Heavy': 3}
#             df['Traffic_Numeric'] = df['Traffic'].map(traffic_map).fillna(0).astype(np.int8)
#             df['Distance_Traffic'] = df['Distance_KM'] * df['Traffic_Numeric']
    
#     print(f"Feature engineering completed. Shape: {df.shape}")
#     return df
    
# config.USE_TEMPORAL = True
# config.USE_DISTANCE = True
# config.USE_SENSOR_AGG = True
# config.USE_ECONOMIC = True
# config.USE_INTERACTION = True

# train = engineer_features(train, is_train=True)
# test = engineer_features(test, is_train=False)

# gc.collect()

Engineering features for train set...
  - Creating temporal features...
  - Creating distance features...
  - Creating sensor aggregation features...
  - Creating economic features...
  - Creating interaction features...
Feature engineering completed. Shape: (8000000, 55)
Engineering features for test set...
  - Creating temporal features...
  - Creating distance features...
  - Creating sensor aggregation features...
  - Creating economic features...
  - Creating interaction features...
Feature engineering completed. Shape: (4000000, 54)


0

In [6]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    delta_lat = np.radians(lat2 - lat1)
    delta_lon = np.radians(lon2 - lon1)
    a = np.sin(delta_lat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

def bearing(lat1, lon1, lat2, lon2):
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    diff = np.radians(lon2 - lon1)
    x = np.sin(diff) * np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(diff)
    return np.degrees(np.arctan2(x, y))

def geo_bin(lat, lon, precision=2):
    return lat.round(precision).astype(str) + '_' + lon.round(precision).astype(str)

def engineer_features(df, is_train=True):
    print(f"Engineering features for {'train' if is_train else 'test'} set...")
    df = df.copy()

    df['null_count'] = df.isnull().sum(axis=1)

    if config.USE_TEMPORAL and 'Timestamp' in df.columns:
        temporal_cols = ['Timestamp']
        df['null_count_temporal'] = df[temporal_cols].isnull().sum(axis=1)

        df['Timestamp_parsed'] = pd.to_datetime(df['Timestamp'], errors='coerce')
        df['Hour'] = df['Timestamp_parsed'].dt.hour
        df['DayOfWeek'] = df['Timestamp_parsed'].dt.dayofweek
        df['Month'] = df['Timestamp_parsed'].dt.month
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(float)
        df['IsRushHour'] = ((df['Hour'] >= 7) & (df['Hour'] <= 9) | 
                            (df['Hour'] >= 17) & (df['Hour'] <= 19)).astype(float)
        df['IsLateNight'] = ((df['Hour'] >= 22) | (df['Hour'] <= 5)).astype(float)
        df.drop('Timestamp_parsed', axis=1, inplace=True)

    if config.USE_DISTANCE:
        distance_cols = [c for c in ['Pickup_Lat', 'Pickup_Long', 'Dropoff_Lat', 'Dropoff_Long', 'Distance_KM'] if c in df.columns]
        if distance_cols:
            df['null_count_distance'] = df[distance_cols].isnull().sum(axis=1)

        if all(col in df.columns for col in ['Pickup_Lat', 'Pickup_Long', 'Dropoff_Lat', 'Dropoff_Long']):
            df['Haversine_Distance'] = haversine_distance(
                df['Pickup_Lat'], df['Pickup_Long'],
                df['Dropoff_Lat'], df['Dropoff_Long']
            )
            df['Delta_Lat'] = df['Dropoff_Lat'] - df['Pickup_Lat']
            df['Bearing'] = bearing(df.Pickup_Lat, df.Pickup_Long, df.Dropoff_Lat, df.Dropoff_Long)
            df["bearing_sin"] = np.sin(np.radians(df["Bearing"])) 
            df["bearing_cos"] = np.cos(np.radians(df["Bearing"]))
            df['pickup_grid'] = geo_bin(df['Pickup_Lat'], df['Pickup_Long'])

            if 'Distance_KM' in df.columns:
                df['Distance_Ratio'] = df['Distance_KM'] / (df['Haversine_Distance'] + 1e-6)
                df['Distance_Difference'] = np.abs(df['Distance_KM'] - df['Haversine_Distance'])
                df['Is_Ultra_Short_Distance'] = (df['Distance_KM'] < 0.024).astype(int)

        if 'Pickup_Zone' in df.columns and 'Dropoff_Zone' in df.columns:
            df['Is_Same_Zone'] = (df['Pickup_Zone'] == df['Dropoff_Zone']).astype(np.int8)
            pickup_freq = df['Pickup_Zone'].value_counts()
            dropoff_freq = df['Dropoff_Zone'].value_counts()
            df['pickup_zone_count'] = df['Pickup_Zone'].map(pickup_freq).fillna(0)
            df['dropoff_zone_count'] = df['Dropoff_Zone'].map(dropoff_freq).fillna(0)

    if config.USE_SENSOR_AGG:
        sensor_cols = [c for c in ['Accel_X', 'Accel_Y', 'Accel_Z', 'Gyro_Z'] if c in df.columns]
        if sensor_cols:
            df['null_count_sensor_agg'] = df[sensor_cols].isnull().sum(axis=1)

        if all(col in df.columns for col in ['Accel_X', 'Accel_Y', 'Accel_Z']):
            df['Accel_Magnitude'] = np.sqrt(df['Accel_X']**2 + df['Accel_Y']**2 + df['Accel_Z']**2)
            df['Accel_Max'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].max(axis=1)
            df['Accel_Min'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].min(axis=1)
            df['Accel_Std'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].std(axis=1)
            df['Accel_Range'] = df['Accel_Max'] - df['Accel_Min']
            df['Is_Accel_Magnitude_Outlier'] = (df['Accel_Magnitude'].abs() >= 10.80)

        if 'Gyro_Z' in df.columns:
            df['Gyro_Abs'] = np.abs(df['Gyro_Z'])
            df['Is_Gyro_Z_Outlier'] = (df['Gyro_Z'].abs() >= 0.672).astype(int)

    if config.USE_ECONOMIC:
        econ_cols = [c for c in ['Est_Price_IDR', 'Distance_KM', 'Promo_Code', 'Surge_Multiplier'] if c in df.columns]
        if econ_cols:
            df['null_count_economic'] = df[econ_cols].isnull().sum(axis=1)

        if 'Est_Price_IDR' in df.columns and 'Distance_KM' in df.columns:
            df['Price_per_KM'] = df['Est_Price_IDR'] / (df['Distance_KM'] + 1e-6)

        def encode_promo(x):
            if pd.isna(x):
                return 0
            if x == 'NO VOUCHER':
                return 1
            return 2

        if 'Promo_Code' in df.columns:
            df['Promo_Code_enc'] = df['Promo_Code'].apply(encode_promo).astype(np.int8)

        if 'Surge_Multiplier' in df.columns:
            surge_filled = df['Surge_Multiplier'].fillna(1.0)
            df['Surge_Category'] = pd.cut(
                surge_filled,
                bins=[0, 1, 1.5, 2, 10],
                labels=[0, 1, 2, 3]
            ).astype(np.int8)

    if config.USE_INTERACTION:
        interaction_cols = [c for c in ['Surge_Multiplier', 'Hour', 'Distance_KM', 'Traffic'] if c in df.columns]
        if interaction_cols:
            df['null_count_interaction'] = df[interaction_cols].isnull().sum(axis=1)

        if 'Surge_Multiplier' in df.columns and 'Hour' in df.columns:
            df['Surge_Hour_Interaction'] = df['Surge_Multiplier'] * df['Hour']

        if 'Distance_KM' in df.columns and 'Traffic' in df.columns:
            traffic_map = {'Light': 1, 'Moderate': 2, 'Heavy': 3}
            df['Traffic_Numeric'] = df['Traffic'].map(traffic_map).fillna(0).astype(np.int8)
            df['Distance_Traffic'] = df['Distance_KM'] * df['Traffic_Numeric']

    print(f"Feature engineering completed. Shape: {df.shape}")
    return df

config.USE_TEMPORAL = True
config.USE_DISTANCE = True
config.USE_SENSOR_AGG = True
config.USE_ECONOMIC = True
config.USE_INTERACTION = True

train = engineer_features(train, is_train=True)
test = engineer_features(test, is_train=False)

gc.collect()


Engineering features for train set...
Feature engineering completed. Shape: (8000000, 63)
Engineering features for test set...
Feature engineering completed. Shape: (4000000, 62)


0

In [8]:
# def haversine_distance(lat1, lon1, lat2, lon2):
#     R = 6371
#     lat1_rad = np.radians(lat1)
#     lat2_rad = np.radians(lat2)
#     delta_lat = np.radians(lat2 - lat1)
#     delta_lon = np.radians(lon2 - lon1)

#     a = np.sin(delta_lat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(delta_lon/2)**2
#     c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

#     return R * c

# def bearing(lat1, lon1, lat2, lon2):
#     lat1, lat2 = np.radians(lat1), np.radians(lat2)
#     diff = np.radians(lon2 - lon1)
#     x = np.sin(diff) * np.cos(lat2)
#     y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(diff)
#     return np.degrees(np.arctan2(x, y))

# def geo_bin(lat, lon, precision=2):
#     return lat.round(precision).astype(str) + '_' + lon.round(precision).astype(str)

# def engineer_features(df, is_train=True):
#     print(f"Engineering features for {'train' if is_train else 'test'} set...")
#     df = df.copy()

#     df['null_count'] = df.isnull().sum(axis=1)

#     if config.USE_TEMPORAL and 'Timestamp' in df.columns:
#         print("  - Creating temporal features...")
#         temporal_cols = ['Timestamp']
#         df['null_count_temporal'] = df[temporal_cols].isnull().sum(axis=1)
        
#         df['Timestamp_parsed'] = pd.to_datetime(df['Timestamp'], errors='coerce')
#         df['Hour'] = df['Timestamp_parsed'].dt.hour
#         df['DayOfWeek'] = df['Timestamp_parsed'].dt.dayofweek
#         df['Month'] = df['Timestamp_parsed'].dt.month
#         df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(float)
#         df['IsRushHour'] = ((df['Hour'] >= 7) & (df['Hour'] <= 9) | 
#                             (df['Hour'] >= 17) & (df['Hour'] <= 19)).astype(float)
#         df['IsLateNight'] = ((df['Hour'] >= 22) | (df['Hour'] <= 5)).astype(float)
#         df.drop('Timestamp_parsed', axis=1, inplace=True)

#     if config.USE_DISTANCE:
#         print("  - Creating distance features...")
#         distance_cols = [c for c in ['Pickup_Lat', 'Pickup_Long', 'Dropoff_Lat', 'Dropoff_Long', 'Distance_KM'] if c in df.columns]
#         if distance_cols:
#             df['null_count_distance'] = df[distance_cols].isnull().sum(axis=1)
            
#         if all(col in df.columns for col in ['Pickup_Lat', 'Pickup_Long', 'Dropoff_Lat', 'Dropoff_Long']):
#             df['Haversine_Distance'] = haversine_distance(
#                 df['Pickup_Lat'], df['Pickup_Long'],
#                 df['Dropoff_Lat'], df['Dropoff_Long']
#             )
#             df['Delta_Lat'] = df['Dropoff_Lat'] - df['Pickup_Lat']
#             df['Bearing'] = bearing(df.Pickup_Lat, df.Pickup_Long, df.Dropoff_Lat, df.Dropoff_Long)
#             df["bearing_sin"] = np.sin(np.radians(df["Bearing"])) 
#             df["bearing_cos"] = np.cos(np.radians(df["Bearing"]))
#             df['pickup_grid'] = geo_bin(df['Pickup_Lat'], df['Pickup_Long'])

#             if 'Distance_KM' in df.columns:
#                 df['Distance_Ratio'] = df['Distance_KM'] / (df['Haversine_Distance'] + 1e-6)
#                 df['Distance_Difference'] = np.abs(df['Distance_KM'] - df['Haversine_Distance'])
#                 df['Is_Ultra_Short_Distance'] = (df['Distance_KM'] < 0.024).astype(int)

#         if 'Pickup_Zone' in df.columns and 'Dropoff_Zone' in df.columns:
#             df['Is_Same_Zone'] = (df['Pickup_Zone'] == df['Dropoff_Zone']).astype(np.int8)

#             pickup_freq = df['Pickup_Zone'].value_counts()
#             dropoff_freq = df['Dropoff_Zone'].value_counts()

#             df['pickup_zone_count'] = df['Pickup_Zone'].map(pickup_freq).fillna(0)
#             df['dropoff_zone_count'] = df['Dropoff_Zone'].map(dropoff_freq).fillna(0)

#     if config.USE_SENSOR_AGG:
#         print("  - Creating sensor aggregation features...")
#         sensor_cols = [c for c in ['Accel_X', 'Accel_Y', 'Accel_Z', 'Gyro_Z'] if c in df.columns]
#         if sensor_cols:
#             df['null_count_sensor_agg'] = df[sensor_cols].isnull().sum(axis=1)
            
#         if all(col in df.columns for col in ['Accel_X', 'Accel_Y', 'Accel_Z']):
#             df['Accel_Magnitude'] = np.sqrt(df['Accel_X']**2 + df['Accel_Y']**2 + df['Accel_Z']**2)
#             df['Accel_Max'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].max(axis=1)
#             df['Accel_Min'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].min(axis=1)
#             df['Accel_Std'] = df[['Accel_X', 'Accel_Y', 'Accel_Z']].std(axis=1)
#             df['Accel_Range'] = df['Accel_Max'] - df['Accel_Min']
#             df['Is_Accel_Magnitude_Outlier'] = (df['Accel_Magnitude'].abs() >= 10.80)

#         if 'Gyro_Z' in df.columns:
#             df['Gyro_Abs'] = np.abs(df['Gyro_Z'])
#             df['Is_Gyro_Z_Outlier'] = (df['Gyro_Z'].abs() >= 0.672).astype(int)

#     if config.USE_ECONOMIC:
#         print("  - Creating economic features...")
#         econ_cols = [c for c in ['Est_Price_IDR', 'Distance_KM', 'Promo_Code', 'Surge_Multiplier'] if c in df.columns]
#         if econ_cols:
#             df['null_count_economic'] = df[econ_cols].isnull().sum(axis=1)
            
#         if 'Est_Price_IDR' in df.columns and 'Distance_KM' in df.columns:
#             df['Price_per_KM'] = df['Est_Price_IDR'] / (df['Distance_KM'] + 1e-6)

#         def encode_promo(x):
#             if pd.isna(x):
#                 return 0
#             if x == 'NO VOUCHER':
#                 return 1
#             return 2

#         if 'Promo_Code' in df.columns:
#             df['Promo_Code_enc'] = df['Promo_Code'].apply(encode_promo).astype(np.int8)

#         if 'Surge_Multiplier' in df.columns:
#             surge_filled = df['Surge_Multiplier'].fillna(1.0)
#             df['Surge_Category'] = pd.cut(
#                 surge_filled,
#                 bins=[0, 1, 1.5, 2, 10],
#                 labels=[0, 1, 2, 3]
#             ).astype(np.int8)

#     if config.USE_INTERACTION:
#         print("  - Creating interaction features...")
#         interaction_cols = [c for c in ['Surge_Multiplier', 'Hour', 'Distance_KM', 'Traffic'] if c in df.columns]
#         if interaction_cols:
#             df['null_count_interaction'] = df[interaction_cols].isnull().sum(axis=1)
            
#         if 'Surge_Multiplier' in df.columns and 'Hour' in df.columns:
#             df['Surge_Hour_Interaction'] = df['Surge_Multiplier'] * df['Hour']

#         if 'Distance_KM' in df.columns and 'Traffic' in df.columns:
#             traffic_map = {'Light': 1, 'Moderate': 2, 'Heavy': 3}
#             df['Traffic_Numeric'] = df['Traffic'].map(traffic_map).fillna(0).astype(np.int8)
#             df['Distance_Traffic'] = df['Distance_KM'] * df['Traffic_Numeric']

#     print(f"Feature engineering completed. Shape: {df.shape}")
#     return df

# config.USE_TEMPORAL = True
# config.USE_DISTANCE = True
# config.USE_SENSOR_AGG = True
# config.USE_ECONOMIC = True
# config.USE_INTERACTION = True

# train = engineer_features(train, is_train=True)
# test = engineer_features(test, is_train=False)

# pickup_price_mean = train.groupby('Pickup_Zone')['Est_Price_IDR'].mean()
# pickup_dist_mean = train.groupby('Pickup_Zone')['Distance_KM'].mean()

# train['pickup_zone_avg_price'] = train['Pickup_Zone'].map(pickup_price_mean).fillna(0)
# test['pickup_zone_avg_price'] = test['Pickup_Zone'].map(pickup_price_mean).fillna(0)

# train['pickup_zone_avg_distance'] = train['Pickup_Zone'].map(pickup_dist_mean).fillna(0)
# test['pickup_zone_avg_distance'] = test['Pickup_Zone'].map(pickup_dist_mean).fillna(0)

# train_route = train['Pickup_Zone'] + '__' + train['Dropoff_Zone']
# test_route = test['Pickup_Zone'] + '__' + test['Dropoff_Zone']

# route_freq = train_route.value_counts()

# train['route_count'] = train_route.map(route_freq).fillna(0)
# test['route_count'] = test_route.map(route_freq).fillna(0)

# gc.collect()

Engineering features for train set...
  - Creating temporal features...
  - Creating distance features...
  - Creating sensor aggregation features...
  - Creating economic features...
  - Creating interaction features...
Feature engineering completed. Shape: (8000000, 66)
Engineering features for test set...
  - Creating temporal features...
  - Creating distance features...
  - Creating sensor aggregation features...
  - Creating economic features...
  - Creating interaction features...
Feature engineering completed. Shape: (4000000, 65)


102

In [7]:
def encode_categorical_frequency(train, test, categorical_cols):
    """
    Encodes based on value frequency in training data
    """
    print("\n" + "="*80)
    print("FREQUENCY ENCODING")
    print("="*80)
    
    encoders = {}
    
    for col in tqdm(categorical_cols, desc="Frequency encoding"):
        freq_map = train[col].value_counts(dropna=False).to_dict()
        
        train[col] = train[col].map(freq_map).fillna(0).astype(np.int32)
        test[col] = test[col].map(freq_map).fillna(0).astype(np.int32)
        
        encoders[col] = {
            'type': 'frequency',
            'unique_values': len(freq_map),
            'unseen_in_test': (test[col] == 0).sum()
        }
    
    print(f"\nEncoded {len(categorical_cols)} categorical features")
    return train, test, encoders

In [8]:
def encode_categorical_target(train, test, categorical_cols, y_train, smoothing=10):
    """
    Encodes based on target mean, with smoothing to prevent overfitting
    """
    print("\n" + "="*80)
    print("TARGET ENCODING")
    print("="*80)
    
    encoders = {}
    global_mean = y_train.mean()
    
    for col in tqdm(categorical_cols, desc="Target encoding"):
        temp_df = pd.DataFrame({col: train[col], 'target': y_train})
        
        agg = temp_df.groupby(col)['target'].agg(['mean', 'count'])
        smoothed_mean = (agg['mean'] * agg['count'] + global_mean * smoothing) / (agg['count'] + smoothing)
        
        encoding_map = smoothed_mean.to_dict()
        
        train[col] = train[col].map(encoding_map).fillna(global_mean).astype(np.float32)
        test[col] = test[col].map(encoding_map).fillna(global_mean).astype(np.float32)
        
        encoders[col] = {
            'type': 'target',
            'unique_values': len(encoding_map),
            'global_mean': global_mean
        }
    
    print(f"\nEncoded {len(categorical_cols)} categorical features")
    return train, test, encoders

In [9]:
def encode_categorical_label_optimized(train, test, categorical_cols):
    print("\n" + "="*80)
    print("LABEL ENCODING")
    print("="*80)

    label_encoders = {}

    for col in tqdm(categorical_cols, desc="Label encoding"):
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)

        le = LabelEncoder()
        train[col] = le.fit_transform(train[col])

        mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        test[col] = test[col].map(mapping).fillna(-1).astype(int)

        label_encoders[col] = le

    print(f"\nEncoded {len(categorical_cols)} categorical features")
    return train, test, label_encoders

In [10]:
# def preprocess_data(train, test, encoding_method='frequency'):
#     """
#     Main preprocessing pipeline
    
#     Parameters:
#     -----------
#     train : pd.DataFrame
#         Training dataset
#     test : pd.DataFrame
#         Test dataset
#     encoding_method : str
#         Encoding method to use: 'frequency', 'target', or 'label'
#     Returns:
#     --------
#     X_train, X_test, y_train, le_target, encoders
#     """
#     print("\n" + "="*80)
#     print("DATA PREPROCESSING PIPELINE")
#     print("="*80)
#     print(f"Encoding method: {encoding_method.upper()}")
    
#     cols_to_drop = [config.ID_COL, 'Timestamp']
#     if config.TARGET_COL in train.columns:
#         y = train[config.TARGET_COL].copy()
#         cols_to_drop.append(config.TARGET_COL)
#     else:
#         y = None
    
#     cols_to_drop = [col for col in cols_to_drop if col in train.columns]
#     X_train = train.drop(cols_to_drop, axis=1).copy()
#     X_test = test.drop([col for col in cols_to_drop if col in test.columns], axis=1).copy()
    
#     print(f"\nInitial shapes:")
#     print(f"  X_train: {X_train.shape}")
#     print(f"  X_test: {X_test.shape}")
    
#     print(f"\nMissing values before imputation:")
#     train_missing = X_train.isnull().sum()
#     if train_missing.sum() > 0:
#         print(train_missing[train_missing > 0])
#     else:
#         print("  No missing values found")
    
#     numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
#     categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    
#     print(f"\nFeature types detected:")
#     print(f"  Numeric features: {len(numeric_cols)}")
#     print(f"  Categorical features: {len(categorical_cols)}")
    
#     print("\n" + "="*80)
#     print("STEP 1: Missing Value Imputation")
#     print("="*80)
    
#     for col in tqdm(numeric_cols, desc="Imputing numeric features"):
#         if X_train[col].isnull().sum() > 0:
#             median_val = X_train[col].median()
#             X_train[col].fillna(median_val, inplace=True)
#             X_test[col].fillna(median_val, inplace=True)
    
#     for col in tqdm(categorical_cols, desc="Imputing categorical features"):
#         if X_train[col].isnull().sum() > 0:
#             X_train[col].fillna('Unknown', inplace=True)
#             X_test[col].fillna('Unknown', inplace=True)
    
#     print("\n" + "="*80)
#     print("STEP 2: Outlier Clipping")
#     print("="*80)
    
#     for col in tqdm(numeric_cols, desc="Clipping outliers"):
#         q99 = X_train[col].quantile(0.99)
#         q01 = X_train[col].quantile(0.01)
#         X_train[col] = X_train[col].clip(q01, q99)
#         X_test[col] = X_test[col].clip(q01, q99)
    
#     print("\n" + "="*80)
#     print("STEP 3: Categorical Encoding")
#     print("="*80)
    
#     if len(categorical_cols) > 0:
#         if encoding_method == 'frequency':
#             X_train, X_test, encoders = encode_categorical_frequency(
#                 X_train, X_test, categorical_cols
#             )
#         elif encoding_method == 'target':
#             if y is None:
#                 raise ValueError("Target encoding requires target variable")
#             le_target_temp = LabelEncoder()
#             y_temp = le_target_temp.fit_transform(y)
#             X_train, X_test, encoders = encode_categorical_target(
#                 X_train, X_test, categorical_cols, y_temp
#             )
#         elif encoding_method == 'label':
#             X_train, X_test, encoders = encode_categorical_label_optimized(
#                 X_train, X_test, categorical_cols
#             )
#         else:
#             raise ValueError(f"Unknown encoding method: {encoding_method}")
#     else:
#         encoders = {}
#         print("No categorical features to encode")
    
#     print("\n" + "="*80)
#     print("STEP 4: Target Encoding")
#     print("="*80)
    
#     if y is not None:
#         le_target = LabelEncoder()
#         y_encoded = le_target.fit_transform(y)
#         print(f"\nTarget classes encoded:")
#         for i, label in enumerate(le_target.classes_):
#             count = (y_encoded == i).sum()
#             print(f"  {i}: {label:20s} - {count:,} samples ({count/len(y_encoded)*100:.2f}%)")
#     else:
#         y_encoded = None
#         le_target = None
    
#     print("\n" + "="*80)
#     print("PREPROCESSING COMPLETED")
#     print("="*80)
#     print(f"Final shapes:")
#     print(f"  X_train: {X_train.shape}")
#     print(f"  X_test: {X_test.shape}")
#     if y_encoded is not None:
#         print(f"  y_train: {y_encoded.shape}")
#     print("="*80)
    
#     return X_train, X_test, y_encoded, le_target, encoders

# X_train, X_test, y_train, le_target, encoders = preprocess_data(train, test, encoding_method='target')
# gc.collect()

In [10]:
def preprocess_data(train, test, encoding_method='frequency'):
    """
    Main preprocessing pipeline
    
    Parameters:
    -----------
    train : pd.DataFrame
        Training dataset
    test : pd.DataFrame
        Test dataset
    encoding_method : str
        Encoding method to use: 'frequency', 'target', or 'label'
    Returns:
    --------
    X_train, X_test, y_train, le_target, encoders
    """
    print("\n" + "="*80)
    print("DATA PREPROCESSING PIPELINE")
    print("="*80)
    print(f"Encoding method: {encoding_method.upper()}")
    
    cols_to_drop = [config.ID_COL, 'Timestamp']
    if config.TARGET_COL in train.columns:
        y = train[config.TARGET_COL].copy()
        cols_to_drop.append(config.TARGET_COL)
    else:
        y = None
    
    cols_to_drop = [col for col in cols_to_drop if col in train.columns]
    X_train = train.drop(cols_to_drop, axis=1).copy()
    X_test = test.drop([col for col in cols_to_drop if col in test.columns], axis=1).copy()
    
    print(f"\nInitial shapes:")
    print(f"  X_train: {X_train.shape}")
    print(f"  X_test: {X_test.shape}")
    
    print(f"\nMissing values before imputation:")
    train_missing = X_train.isnull().sum()
    if train_missing.sum() > 0:
        print(train_missing[train_missing > 0])
    else:
        print("  No missing values found")
    
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    
    print(f"\nFeature types detected:")
    print(f"  Numeric features: {len(numeric_cols)}")
    print(f"  Categorical features: {len(categorical_cols)}")
    
    print("\n" + "="*80)
    print("STEP 1: Missing Value Imputation")
    print("="*80)
    
    mean_impute_cols = ['Accel_Y', 'Accel_Z', 'Dropoff_Lat', 'Dropoff_Long', 'Gyro_Z']
    mean_impute_cols = [c for c in mean_impute_cols if c in numeric_cols]
    median_impute_cols = [c for c in numeric_cols if c not in mean_impute_cols]
    
    for col in tqdm(mean_impute_cols, desc="Imputing numeric features (mean)"):
        mean_val = X_train[col].mean()
        X_train[col].fillna(mean_val, inplace=True)
        X_test[col].fillna(mean_val, inplace=True)
    
    for col in tqdm(median_impute_cols, desc="Imputing numeric features (median)"):
        if X_train[col].isnull().sum() > 0:
            median_val = X_train[col].median()
            X_train[col].fillna(median_val, inplace=True)
            X_test[col].fillna(median_val, inplace=True)
    
    for col in tqdm(categorical_cols, desc="Imputing categorical features"):
        if X_train[col].isnull().sum() > 0:
            X_train[col].fillna('Unknown', inplace=True)
            X_test[col].fillna('Unknown', inplace=True)
    
    print("\n" + "="*80)
    print("STEP 2: Outlier Clipping")
    print("="*80)
    
    for col in tqdm(numeric_cols, desc="Clipping outliers"):
        q99 = X_train[col].quantile(0.99)
        q01 = X_train[col].quantile(0.01)
        X_train[col] = X_train[col].clip(q01, q99)
        X_test[col] = X_test[col].clip(q01, q99)
    
    print("\n" + "="*80)
    print("STEP 3: Categorical Encoding")
    print("="*80)
    
    encoders = {}
    
    ordinal_cols = ['Weather', 'Traffic', 'Payment_Method', 'Signal_Strength', 'Car_Model']
    ordinal_cols = [c for c in ordinal_cols if c in categorical_cols]
    other_cat_cols = [c for c in categorical_cols if c not in ordinal_cols]
    
    if len(ordinal_cols) > 0:
        from sklearn.preprocessing import OrdinalEncoder
        oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        X_train[ordinal_cols] = oe.fit_transform(X_train[ordinal_cols])
        X_test[ordinal_cols] = oe.transform(X_test[ordinal_cols])
        encoders['ordinal'] = oe
    
    if len(other_cat_cols) > 0:
        if encoding_method == 'frequency':
            X_train, X_test, enc = encode_categorical_frequency(
                X_train, X_test, other_cat_cols
            )
            encoders.update(enc)
        elif encoding_method == 'target':
            if y is None:
                raise ValueError("Target encoding requires target variable")
            le_target_temp = LabelEncoder()
            y_temp = le_target_temp.fit_transform(y)
            X_train, X_test, enc = encode_categorical_target(
                X_train, X_test, other_cat_cols, y_temp
            )
            encoders.update(enc)
        elif encoding_method == 'label':
            X_train, X_test, enc = encode_categorical_label_optimized(
                X_train, X_test, other_cat_cols
            )
            encoders.update(enc)
        else:
            raise ValueError(f"Unknown encoding method: {encoding_method}")
    
    print("\n" + "="*80)
    print("STEP 4: Target Encoding")
    print("="*80)
    
    if y is not None:
        le_target = LabelEncoder()
        y_encoded = le_target.fit_transform(y)
        print(f"\nTarget classes encoded:")
        for i, label in enumerate(le_target.classes_):
            count = (y_encoded == i).sum()
            print(f"  {i}: {label:20s} - {count:,} samples ({count/len(y_encoded)*100:.2f}%)")
    else:
        y_encoded = None
        le_target = None
    
    print("\n" + "="*80)
    print("PREPROCESSING COMPLETED")
    print("="*80)
    print(f"Final shapes:")
    print(f"  X_train: {X_train.shape}")
    print(f"  X_test: {X_test.shape}")
    if y_encoded is not None:
        print(f"  y_train: {y_encoded.shape}")
    print("="*80)
    
    return X_train, X_test, y_encoded, le_target, encoders

X_train, X_test, y_train, le_target, encoders = preprocess_data(train, test, encoding_method='target')
gc.collect()


DATA PREPROCESSING PIPELINE
Encoding method: TARGET

Initial shapes:
  X_train: (8000000, 60)
  X_test: (4000000, 60)

Missing values before imputation:
Pickup_Lat                 929348
Pickup_Long                611083
Dropoff_Lat               1914440
Dropoff_Long              1556029
GPS_Accuracy_M            1504090
Distance_KM                941317
Est_Price_IDR             1151401
Surge_Multiplier           612307
Accel_X                   1608442
Accel_Y                   1081337
Accel_Z                   1840229
Gyro_Z                     701714
Pickup_Zone               1381274
Dropoff_Zone               765445
Device_FP                 1499497
Promo_Code                1107810
Car_Model                 1721777
Payment_Method            1141190
Weather                    514649
Traffic                   1808403
Battery_Level             1746399
Signal_Strength            408588
Hour                      1637893
DayOfWeek                 1637893
Month                     1637

Imputing numeric features (mean):   0%|          | 0/5 [00:00<?, ?it/s]

Imputing numeric features (median):   0%|          | 0/43 [00:00<?, ?it/s]

Imputing categorical features:   0%|          | 0/11 [00:00<?, ?it/s]


STEP 2: Outlier Clipping


Clipping outliers:   0%|          | 0/48 [00:00<?, ?it/s]


STEP 3: Categorical Encoding

TARGET ENCODING


Target encoding:   0%|          | 0/6 [00:00<?, ?it/s]


Encoded 6 categorical features

STEP 4: Target Encoding

Target classes encoded:
  0: Fraud_Indication     - 400,313 samples (5.00%)
  1: Navigation_Issue     - 801,790 samples (10.02%)
  2: Perfect_Trip         - 4,397,607 samples (54.97%)
  3: Safety_Violation     - 1,601,595 samples (20.02%)
  4: Service_Complaint    - 798,695 samples (9.98%)

PREPROCESSING COMPLETED
Final shapes:
  X_train: (8000000, 60)
  X_test: (4000000, 60)
  y_train: (8000000,)


80

In [11]:
def macro_f1_eval(preds, dtrain):
    """
    Custom evaluation function for XGBoost to calculate Macro F1
    """
    labels = dtrain.get_label()
    preds_reshaped = preds.reshape(len(labels), -1)
    pred_labels = np.argmax(preds_reshaped, axis=1)
    # Calculate macro F1
    score = f1_score(labels, pred_labels, average='macro')
    return 'macro_f1', score

In [12]:
def macro_f1_lgb(preds, dataset):
    y_true = dataset.get_label()
    num_class = len(np.unique(y_true))
    preds_reshaped = preds.reshape(num_class, -1).T  # shape (n_samples, n_classes)
    y_pred_labels = np.argmax(preds_reshaped, axis=1)
    score = f1_score(y_true, y_pred_labels, average='macro')
    return 'macro_f1', score, True  # True -> higher is better

# Tuning

In [25]:
import optuna
from tqdm import trange

def tune_class_weights_optuna(X, y, X_test, n_trials=30):
    def objective(trial):
        w4 = trial.suggest_float("w4", 1.0, 10.0)
        w1 = trial.suggest_float("w1", 1.0, 10.0)

        sample_weight = np.ones(len(y))
        sample_weight[y == 4] = w4
        sample_weight[y == 1] = w1

        _, _, score = train_xgboost(
            X,
            y,
            X_test,
            use_gpu=gpu_config['xgboost_gpu'],
            sample_weight=sample_weight
        )

        print(f"Trial F1 Macro: {score:.6f}")
        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Best Params:", study.best_params)
    print("Best F1:", study.best_value)
    return study.best_params

In [39]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False
    
def train_xgboost(X_train, y_train, X_test, use_gpu=False, sample_weight=None):
    if not XGBOOST_AVAILABLE:
        return None, None, None

    params = config.get_xgboost_params(use_gpu=use_gpu)
    params['num_class'] = len(np.unique(y_train))

    X_tr, X_val, y_tr, y_val, w_tr, w_val = train_test_split(
        X_train,
        y_train,
        sample_weight,
        test_size=0.2,
        stratify=y_train,
        random_state=config.RANDOM_STATE
    )

    dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dval = xgb.DMatrix(X_val, label=y_val, weight=w_val)
    dtest = xgb.DMatrix(X_test)

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dval, 'valid')],
        custom_metric=macro_f1_eval,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    val_preds = model.predict(dval)
    val_labels = np.argmax(val_preds, axis=1)
    score = f1_score(y_val, val_labels, average='macro')

    test_preds = model.predict(dtest)

    return test_preds, model, score


In [31]:
best_weights = tune_class_weights_optuna(X_train, y_train, X_test, n_trials=30)

[I 2026-01-13 16:48:37,926] A new study created in memory with name: no-name-d54e8e47-5e2c-47bd-9952-fb36689a68f9


  XGBoost: GPU mode activated


[I 2026-01-13 16:49:26,090] Trial 0 finished with value: 0.5048238160889713 and parameters: {'w4': 8.786316835058415, 'w1': 5.128610516284762}. Best is trial 0 with value: 0.5048238160889713.


Trial F1 Macro: 0.504824
  XGBoost: GPU mode activated


[I 2026-01-13 16:50:13,175] Trial 1 finished with value: 0.5092455724423084 and parameters: {'w4': 8.04211571680899, 'w1': 3.367347839281938}. Best is trial 1 with value: 0.5092455724423084.


...

Trial F1 Macro: 0.506546
  XGBoost: GPU mode activated


[I 2026-01-13 17:11:52,887] Trial 28 finished with value: 0.577616994034096 and parameters: {'w4': 5.750477388290963, 'w1': 5.081686468357142}. Best is trial 26 with value: 0.632728986733007.


Trial F1 Macro: 0.577617
  XGBoost: GPU mode activated


[I 2026-01-13 17:12:41,160] Trial 29 finished with value: 0.6330866554432486 and parameters: {'w4': 4.863114750907993, 'w1': 5.305772729010983}. Best is trial 29 with value: 0.6330866554432486.


Trial F1 Macro: 0.633087
Best Params: {'w4': 4.863114750907993, 'w1': 5.305772729010983}
Best F1: 0.6330866554432486


In [32]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

def train_xgboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not XGBOOST_AVAILABLE:
        return None, None, None, None

    print("\n" + "="*80)
    print("Training XGBoost Models")
    print("="*80)

    params = config.get_xgboost_params(use_gpu=use_gpu)
    params['num_class'] = len(np.unique(y_train))

    sample_weight = np.ones(len(y_train))
    sample_weight[y_train == 4] = best_weights["w4"]
    sample_weight[y_train == 1] = best_weights["w1"]

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=config.RANDOM_STATE)

    oof_predictions = np.zeros((len(X_train), len(np.unique(y_train))))
    test_predictions = np.zeros((len(X_test), len(np.unique(y_train))))

    fold_scores = []
    models = []

    pbar = tqdm(enumerate(skf.split(X_train, y_train), 1), total=n_folds, desc="XGBoost Folds")
    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"XGBoost Fold {fold}/{n_folds}")

        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=sample_weight[train_idx])
        dval = xgb.DMatrix(X_val, label=y_val, weight=sample_weight[val_idx])
        dtest = xgb.DMatrix(X_test)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dtrain, 'train'), (dval, 'valid')],
            custom_metric=macro_f1_eval,
            early_stopping_rounds=50,
            verbose_eval=100
        )

        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / n_folds

        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average='macro')
        fold_scores.append(fold_score)

        print(f"\n{'='*60}")
        print(f"FOLD {fold} SUMMARY:")
        print(f"{'='*60}")
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.best_iteration}")
        print(f"  Best Score (mlogloss): {model.best_score:.6f}")

        oof_fold_pred = np.argmax(oof_predictions[val_idx], axis=1)
        print(f"\n  Per-Class F1 Scores:")
        from sklearn.metrics import classification_report
        print(classification_report(
            y_val,
            oof_fold_pred,
            target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
            digits=4
        ))
        print(f"{'='*60}\n")

        pbar.set_postfix({'F1': f'{fold_score:.6f}'})

        models.append(model)
        gc.collect()

    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average='macro')

    print("\n" + "="*80)
    print("XGBOOST TRAINING SUMMARY")
    print("="*80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")
    print(f"\nFold-by-Fold Scores:")
    for i, score in enumerate(fold_scores, 1):
        print(f"  Fold {i}: {score:.6f}")

    print(f"\n" + "="*80)
    print("OVERALL OUT-OF-FOLD PREDICTIONS REPORT")
    print("="*80)
    print(classification_report(
        y_train,
        oof_pred_labels,
        target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
        digits=4
    ))
    print("="*80)

    return oof_predictions, test_predictions, models, overall_score


if XGBOOST_AVAILABLE:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = train_xgboost(
        X_train,
        y_train,
        X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config['xgboost_gpu']
    )
else:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = None, None, None, 0.0


Training XGBoost Models
  XGBoost: GPU mode activated


XGBoost Fold 1/5:   0%|          | 0/5 [00:01<?, ?it/s]

[0]	train-mlogloss:1.51950	train-macro_f1:0.20129	valid-mlogloss:1.51950	valid-macro_f1:0.20148
[50]	train-mlogloss:1.01235	train-macro_f1:0.63319	valid-mlogloss:1.01237	valid-macro_f1:0.63343

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.633434
  Best Iteration: 0
  Best Score (mlogloss): 0.201484

  Per-Class F1 Scores:


XGBoost Fold 2/5:  20%|██        | 1/5 [01:14<04:59, 74.82s/it, F1=0.633434]

              precision    recall  f1-score   support

     Class_0     0.9971    0.9794    0.9881     80062
     Class_1     0.2147    0.1879    0.2004    160358
     Class_2     0.7332    0.8257    0.7767    879522
     Class_3     0.9956    0.9486    0.9715    320319
     Class_4     0.3310    0.1767    0.2304    159739

    accuracy                         0.7293   1600000
   macro avg     0.6543    0.6237    0.6334   1600000
weighted avg     0.7068    0.7293    0.7140   1600000


[0]	train-mlogloss:1.51949	train-macro_f1:0.20136	valid-mlogloss:1.51948	valid-macro_f1:0.20117
[50]	train-mlogloss:1.01241	train-macro_f1:0.63326	valid-mlogloss:1.01224	valid-macro_f1:0.63322

FOLD 2 SUMMARY:
  Validation F1 (Macro): 0.633217
  Best Iteration: 0
  Best Score (mlogloss): 0.201174

  Per-Class F1 Scores:


...

XGBoost Fold 5/5:  80%|████████  | 4/5 [04:53<01:12, 72.98s/it, F1=0.632940]

              precision    recall  f1-score   support

     Class_0     0.9972    0.9808    0.9890     80063
     Class_1     0.2132    0.1921    0.2021    160358
     Class_2     0.7327    0.8217    0.7747    879521
     Class_3     0.9957    0.9493    0.9719    320319
     Class_4     0.3265    0.1740    0.2271    159739

    accuracy                         0.7274   1600000
   macro avg     0.6531    0.6236    0.6329   1600000
weighted avg     0.7060    0.7274    0.7128   1600000


[0]	train-mlogloss:1.51950	train-macro_f1:0.20148	valid-mlogloss:1.51949	valid-macro_f1:0.20133
[50]	train-mlogloss:1.01230	train-macro_f1:0.63278	valid-mlogloss:1.01230	valid-macro_f1:0.63225

FOLD 5 SUMMARY:
  Validation F1 (Macro): 0.632254
  Best Iteration: 0
  Best Score (mlogloss): 0.201330

  Per-Class F1 Scores:


XGBoost Fold 5/5: 100%|██████████| 5/5 [06:06<00:00, 73.35s/it, F1=0.632254]

              precision    recall  f1-score   support

     Class_0     0.9975    0.9796    0.9885     80063
     Class_1     0.2018    0.2129    0.2072    160358
     Class_2     0.7330    0.8009    0.7654    879521
     Class_3     0.9958    0.9487    0.9717    320319
     Class_4     0.3263    0.1758    0.2285    159739

    accuracy                         0.7181   1600000
   macro avg     0.6509    0.6236    0.6323   1600000
weighted avg     0.7050    0.7181    0.7083   1600000





XGBOOST TRAINING SUMMARY
Overall CV Score (Macro F1): 0.633021
Standard Deviation: 0.000398
Min F1 Score: 0.632254
Max F1 Score: 0.633434

Fold-by-Fold Scores:
  Fold 1: 0.633434
  Fold 2: 0.633217
  Fold 3: 0.632933
  Fold 4: 0.632940
  Fold 5: 0.632254

OVERALL OUT-OF-FOLD PREDICTIONS REPORT
              precision    recall  f1-score   support

     Class_0     0.9972    0.9797    0.9884    400313
     Class_1     0.2127    0.1915    0.2016    801790
     Class_2     0.7330    0.8219    0.7749   4397607
     Class_3     0.9958    0.9489    0.9717   1601595
     Class_4     0.3279    0.1754    0.2285    798695

    accuracy                         0.7275   8000000
   macro avg     0.6533    0.6235    0.6330   8000000
weighted avg     0.7062    0.7275    0.7130   8000000



In [35]:
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80)

if test_predictions is not None:
    print(f"\n✓ XGBoost Model Successfully Trained")
    print(f"  Cross-Validation Score: {xgboost_cv_score:.6f}")
    print(f"  Model Type: XGBoost with GPU acceleration")

    print("\n" + "="*80)
    print("GENERATING PREDICTIONS ON TEST SET")
    print("="*80)

    final_predictions = test_predictions
    final_pred_labels = np.argmax(final_predictions, axis=1)

    print(f"\n✓ Predictions Generated Successfully")
    print(f"  Total test samples: {len(final_pred_labels):,}")
    print(f"  Prediction shape: {final_predictions.shape}")
    print(f"  Classes predicted: {len(np.unique(final_pred_labels))}")

    print("\n" + "="*80)
    print("PREDICTION DISTRIBUTION")
    print("="*80)

    unique, counts = np.unique(final_pred_labels, return_counts=True)
    for class_idx, count in zip(unique, counts):
        percentage = (count / len(final_pred_labels)) * 100
        print(f"  Class {class_idx}: {count:,} samples ({percentage:.2f}%)")

else:
    raise ValueError("XGBoost model training failed! Cannot generate predictions.")

print("\n" + "="*80)



MODEL EVALUATION

✓ XGBoost Model Successfully Trained
  Cross-Validation Score: 0.633021
  Model Type: XGBoost with GPU acceleration

GENERATING PREDICTIONS ON TEST SET

✓ Predictions Generated Successfully
  Total test samples: 4,000,000
  Prediction shape: (4000000, 5)
  Classes predicted: 5

PREDICTION DISTRIBUTION
  Class 0: 198,096 samples (4.95%)
  Class 1: 357,672 samples (8.94%)
  Class 2: 2,424,532 samples (60.61%)
  Class 3: 790,105 samples (19.75%)
  Class 4: 229,595 samples (5.74%)



In [36]:
def create_submission(test_ids, predictions, le_target, filename='submission.csv'):
    pred_labels = le_target.inverse_transform(predictions)
    
    submission = pd.DataFrame({
        config.ID_COL: test_ids,
        config.TARGET_COL: pred_labels
    })
    
    submission.to_csv(filename, index=False)
    
    print(f"\nSubmission saved to: {filename}")
    print(f"Submission shape: {submission.shape}")
    print(f"\nPrediction distribution:")
    print(submission[config.TARGET_COL].value_counts())
    
    return submission

test_ids = test[config.ID_COL].values
submission = create_submission(test_ids, final_pred_labels, le_target, 'anava_deira_9.csv')
submission


Submission saved to: anava_deira_9.csv
Submission shape: (4000000, 2)

Prediction distribution:
Trip_Label
Perfect_Trip         2424532
Safety_Violation      790105
Navigation_Issue      357672
Service_Complaint     229595
Fraud_Indication      198096
Name: count, dtype: int64


,Trip_ID,Trip_Label
0,TRIP-06583736,Perfect_Trip
1,TRIP-11356251,Perfect_Trip
2,TRIP-03320505,Service_Complaint
3,TRIP-07188814,Perfect_Trip
4,TRIP-06994869,Perfect_Trip
...,...,...
3999995,TRIP-02234490,Navigation_Issue
3999996,TRIP-04304573,Perfect_Trip
3999997,TRIP-10081352,Perfect_Trip
3999998,TRIP-06550635,Perfect_Trip


In [12]:
import optuna
from tqdm import trange

def hyperparameter_tuning(X, y, X_test, n_trials=30):
    def objective(trial):
        w1 = trial.suggest_float("w1", 1.0, 6.0)
        w4 = trial.suggest_float("w4", 1.0, 6.0)

        max_depth = trial.suggest_int("max_depth", 4, 8)
        eta = trial.suggest_float("eta", 0.02, 0.15, log=True)
        subsample = trial.suggest_float("subsample", 0.6, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
        min_child_weight = trial.suggest_float("min_child_weight", 1.0, 10.0)

        sample_weight = np.ones(len(y))
        sample_weight[y == 1] = w1
        sample_weight[y == 4] = w4

        params = config.get_xgboost_params(use_gpu=gpu_config['xgboost_gpu'])
        params.update({
            "max_depth": max_depth,
            "eta": eta,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "min_child_weight": min_child_weight,
            "num_class": len(np.unique(y))
        })

        _, _, score = train_xgboost(
            X,
            y,
            X_test,
            use_gpu=gpu_config['xgboost_gpu'],
            sample_weight=sample_weight,
            override_params=params
        )

        print(
            "F1={:.6f} | ".format(score) +
            " | ".join([f"{k}={v}" for k, v in trial.params.items()])
        )

        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Best Params:", study.best_params)
    print("Best F1:", study.best_value)

    return study.best_params

In [15]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False


def train_xgboost(
    X_train,
    y_train,
    X_test,
    use_gpu=False,
    sample_weight=None,
    override_params=None
):
    if not XGBOOST_AVAILABLE:
        return None, None, None

    # ===== PARAMS =====
    if override_params is not None:
        params = override_params.copy()
    else:
        params = config.get_xgboost_params(use_gpu=use_gpu)

    params['num_class'] = len(np.unique(y_train))

    # ===== SPLIT =====
    if sample_weight is not None:
        X_tr, X_val, y_tr, y_val, w_tr, w_val = train_test_split(
            X_train,
            y_train,
            sample_weight,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
    else:
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
        w_tr = w_val = None

    # ===== DMATRIX =====
    dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dval = xgb.DMatrix(X_val, label=y_val, weight=w_val)
    dtest = xgb.DMatrix(X_test)

    # ===== TRAIN =====
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dval, 'valid')],
        custom_metric=macro_f1_eval,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # ===== VALID SCORE =====
    val_preds = model.predict(dval)
    val_labels = np.argmax(val_preds, axis=1)
    score = f1_score(y_val, val_labels, average='macro')

    # ===== TEST =====
    test_preds = model.predict(dtest)

    return test_preds, model, score


In [18]:
best_params = hyperparameter_tuning(X_train, y_train, X_test, n_trials=30)

[I 2026-01-13 17:47:16,079] A new study created in memory with name: no-name-33e2e3b3-bdf0-4dad-b537-ea0d6574b16e


  XGBoost: GPU mode activated


[I 2026-01-13 17:48:01,812] Trial 0 finished with value: 0.6297697274934515 and parameters: {'w1': 4.489232692154786, 'w4': 3.3421817837598917, 'max_depth': 4, 'eta': 0.12454129916362668, 'subsample': 0.6217949553758664, 'colsample_bytree': 0.992390141665997, 'min_child_weight': 4.957760906440891}. Best is trial 0 with value: 0.6297697274934515.


F1=0.629770 | w1=4.489232692154786 | w4=3.3421817837598917 | max_depth=4 | eta=0.12454129916362668 | subsample=0.6217949553758664 | colsample_bytree=0.992390141665997 | min_child_weight=4.957760906440891
  XGBoost: GPU mode activated


[I 2026-01-13 17:48:46,266] Trial 1 finished with value: 0.6289767995930523 and parameters: {'w1': 4.515337948593942, 'w4': 3.052474988250123, 'max_depth': 4, 'eta': 0.06464710787634322, 'subsample': 0.6019107770034554, 'colsample_bytree': 0.7979694103068885, 'min_child_weight': 1.8980914057654275}. Best is trial 0 with value: 0.6297697274934515.


...

F1=0.632954 | w1=4.125509919066976 | w4=4.8157166363206425 | max_depth=8 | eta=0.14990544316136173 | subsample=0.6626619925016584 | colsample_bytree=0.6677574359369572 | min_child_weight=2.5485444205479855
  XGBoost: GPU mode activated


[I 2026-01-13 18:11:25,726] Trial 28 finished with value: 0.6333322129825054 and parameters: {'w1': 4.888735399706439, 'w4': 4.872760673174699, 'max_depth': 7, 'eta': 0.10205554754960605, 'subsample': 0.6653469876314761, 'colsample_bytree': 0.7710588843275792, 'min_child_weight': 5.876892547983603}. Best is trial 4 with value: 0.6341884985218628.


F1=0.633332 | w1=4.888735399706439 | w4=4.872760673174699 | max_depth=7 | eta=0.10205554754960605 | subsample=0.6653469876314761 | colsample_bytree=0.7710588843275792 | min_child_weight=5.876892547983603
  XGBoost: GPU mode activated


[I 2026-01-13 18:12:14,617] Trial 29 finished with value: 0.5419904378611194 and parameters: {'w1': 4.637845338910784, 'w4': 5.924073143027234, 'max_depth': 6, 'eta': 0.04411248792729391, 'subsample': 0.6710099547806788, 'colsample_bytree': 0.9938854475442027, 'min_child_weight': 5.543548811313759}. Best is trial 4 with value: 0.6341884985218628.


F1=0.541990 | w1=4.637845338910784 | w4=5.924073143027234 | max_depth=6 | eta=0.04411248792729391 | subsample=0.6710099547806788 | colsample_bytree=0.9938854475442027 | min_child_weight=5.543548811313759
Best Params: {'w1': 5.11828448724458, 'w4': 4.710347234206637, 'max_depth': 7, 'eta': 0.14917133459423576, 'subsample': 0.8487521681236615, 'colsample_bytree': 0.6101253968127661, 'min_child_weight': 1.2757338320852298}
Best F1: 0.6341884985218628


In [20]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False


def train_xgboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not XGBOOST_AVAILABLE:
        return None, None, None, None

    print("\n" + "=" * 80)
    print("Training XGBoost Models")
    print("=" * 80)

    params = config.get_xgboost_params(use_gpu=use_gpu)
    params["num_class"] = len(np.unique(y_train))

    params.update({
        k: v for k, v in best_params.items()
        if k not in ["w1", "w4"]
    })

    sample_weight = np.ones(len(y_train))
    sample_weight[y_train == 1] = best_params["w1"]
    sample_weight[y_train == 4] = best_params["w4"]

    skf = StratifiedKFold(
        n_splits=n_folds,
        shuffle=True,
        random_state=config.RANDOM_STATE
    )

    oof_predictions = np.zeros((len(X_train), params["num_class"]))
    test_predictions = np.zeros((len(X_test), params["num_class"]))

    fold_scores = []
    models = []

    pbar = tqdm(
        enumerate(skf.split(X_train, y_train), 1),
        total=n_folds,
        desc="XGBoost Folds"
    )

    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"XGBoost Fold {fold}/{n_folds}")

        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        dtrain = xgb.DMatrix(
            X_tr,
            label=y_tr,
            weight=sample_weight[train_idx]
        )
        dval = xgb.DMatrix(
            X_val,
            label=y_val,
            weight=sample_weight[val_idx]
        )
        dtest = xgb.DMatrix(X_test)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dtrain, "train"), (dval, "valid")],
            custom_metric=macro_f1_eval,
            early_stopping_rounds=50,
            verbose_eval=100
        )

        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / n_folds

        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average="macro")
        fold_scores.append(fold_score)

        print("\n" + "=" * 60)
        print(f"FOLD {fold} SUMMARY:")
        print("=" * 60)
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.best_iteration}")
        print(f"  Best Score (mlogloss): {model.best_score:.6f}")

        print(
            classification_report(
                y_val,
                oof_pred_labels,
                target_names=[
                    f"Class_{i}"
                    for i in range(params["num_class"])
                ],
                digits=4
            )
        )
        print("=" * 60 + "\n")

        pbar.set_postfix({"F1": f"{fold_score:.6f}"})

        models.append(model)
        gc.collect()

    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average="macro")

    print("\n" + "=" * 80)
    print("XGBOOST TRAINING SUMMARY")
    print("=" * 80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")

    return oof_predictions, test_predictions, models, overall_score


if XGBOOST_AVAILABLE:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = train_xgboost(
        X_train,
        y_train,
        X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config["xgboost_gpu"]
    )
else:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = None, None, None, 0.0


Training XGBoost Models
  XGBoost: GPU mode activated


XGBoost Fold 1/5:   0%|          | 0/5 [00:01<?, ?it/s]

[0]	train-mlogloss:1.52005	train-macro_f1:0.20203	valid-mlogloss:1.52005	valid-macro_f1:0.20241
[49]	train-mlogloss:1.01544	train-macro_f1:0.63411	valid-mlogloss:1.01555	valid-macro_f1:0.63396

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.634002
  Best Iteration: 0
  Best Score (mlogloss): 0.202409


XGBoost Fold 1/5:  20%|██        | 1/5 [01:16<05:05, 76.34s/it, F1=0.634002]

              precision    recall  f1-score   support

     Class_0     0.9982    0.9782    0.9881     80062
     Class_1     0.2245    0.1691    0.1929    160358
     Class_2     0.7340    0.8337    0.7807    879522
     Class_3     0.9968    0.9488    0.9722    320319
     Class_4     0.3127    0.1896    0.2361    159739

    accuracy                         0.7331   1600000
   macro avg     0.6533    0.6239    0.6340   1600000
weighted avg     0.7067    0.7331    0.7161   1600000




...

XGBoost Fold 5/5:  80%|████████  | 4/5 [04:59<01:14, 74.53s/it, F1=0.633806]

              precision    recall  f1-score   support

     Class_0     0.9986    0.9796    0.9890     80063
     Class_1     0.2281    0.1663    0.1924    160358
     Class_2     0.7338    0.8351    0.7812    879521
     Class_3     0.9970    0.9497    0.9728    320319
     Class_4     0.3065    0.1888    0.2337    159739

    accuracy                         0.7337   1600000
   macro avg     0.6528    0.6239    0.6338   1600000
weighted avg     0.7064    0.7337    0.7163   1600000


[0]	train-mlogloss:1.52006	train-macro_f1:0.20204	valid-mlogloss:1.52006	valid-macro_f1:0.20206
[50]	train-mlogloss:1.01236	train-macro_f1:0.63431	valid-mlogloss:1.01247	valid-macro_f1:0.63396

FOLD 5 SUMMARY:
  Validation F1 (Macro): 0.633957
  Best Iteration: 0
  Best Score (mlogloss): 0.202062


XGBoost Fold 5/5: 100%|██████████| 5/5 [06:15<00:00, 75.01s/it, F1=0.633957]

              precision    recall  f1-score   support

     Class_0     0.9985    0.9782    0.9883     80063
     Class_1     0.2268    0.1682    0.1932    160358
     Class_2     0.7342    0.8326    0.7803    879521
     Class_3     0.9969    0.9492    0.9725    320319
     Class_4     0.3057    0.1916    0.2356    159739

    accuracy                         0.7327   1600000
   macro avg     0.6524    0.6240    0.6340   1600000
weighted avg     0.7064    0.7327    0.7159   1600000





XGBOOST TRAINING SUMMARY
Overall CV Score (Macro F1): 0.633920
Standard Deviation: 0.000173
Min F1 Score: 0.633639
Max F1 Score: 0.634144


In [21]:
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80)

if test_predictions is not None:
    print("\n✓ XGBoost Model Successfully Trained")
    print(f"  Cross-Validation Score: {xgboost_cv_score:.6f}")
    print(f"  Model Type: XGBoost with GPU acceleration")

    print("\n" + "="*80)
    print("GENERATING PREDICTIONS ON TEST SET")
    print("="*80)

    final_predictions = test_predictions
    final_pred_labels = np.argmax(final_predictions, axis=1)

    print("\n✓ Predictions Generated Successfully")
    print(f"  Total test samples: {len(final_pred_labels):,}")
    print(f"  Prediction shape: {final_predictions.shape}")
    print(f"  Classes predicted: {len(np.unique(final_pred_labels))}")

    print("\n" + "="*80)
    print("PREDICTION DISTRIBUTION")
    print("="*80)

    unique, counts = np.unique(final_pred_labels, return_counts=True)
    for class_idx, count in zip(unique, counts):
        percentage = (count / len(final_pred_labels)) * 100
        print(f"  Class {class_idx}: {count:,} samples ({percentage:.2f}%)")
else:
    raise ValueError("XGBoost model training failed! Cannot generate predictions.")

print("\n" + "="*80)



MODEL EVALUATION

✓ XGBoost Model Successfully Trained
  Cross-Validation Score: 0.633920
  Model Type: XGBoost with GPU acceleration

GENERATING PREDICTIONS ON TEST SET

✓ Predictions Generated Successfully
  Total test samples: 4,000,000
  Prediction shape: (4000000, 5)
  Classes predicted: 5

PREDICTION DISTRIBUTION
  Class 0: 197,396 samples (4.93%)
  Class 1: 290,656 samples (7.27%)
  Class 2: 2,458,818 samples (61.47%)
  Class 3: 789,524 samples (19.74%)
  Class 4: 263,606 samples (6.59%)



In [22]:
def create_submission(test_ids, predictions, le_target, filename='submission.csv'):
    pred_labels = le_target.inverse_transform(predictions)
    
    submission = pd.DataFrame({
        config.ID_COL: test_ids,
        config.TARGET_COL: pred_labels
    })
    
    submission.to_csv(filename, index=False)
    
    print(f"\nSubmission saved to: {filename}")
    print(f"Submission shape: {submission.shape}")
    print(f"\nPrediction distribution:")
    print(submission[config.TARGET_COL].value_counts())
    
    return submission

test_ids = test[config.ID_COL].values
submission = create_submission(test_ids, final_pred_labels, le_target, 'anava_deira_10.csv')
submission


Submission saved to: anava_deira_10.csv
Submission shape: (4000000, 2)

Prediction distribution:
Trip_Label
Perfect_Trip         2458818
Safety_Violation      789524
Navigation_Issue      290656
Service_Complaint     263606
Fraud_Indication      197396
Name: count, dtype: int64


,Trip_ID,Trip_Label
0,TRIP-06583736,Perfect_Trip
1,TRIP-11356251,Perfect_Trip
2,TRIP-03320505,Service_Complaint
3,TRIP-07188814,Perfect_Trip
4,TRIP-06994869,Perfect_Trip
...,...,...
3999995,TRIP-02234490,Navigation_Issue
3999996,TRIP-04304573,Perfect_Trip
3999997,TRIP-10081352,Perfect_Trip
3999998,TRIP-06550635,Perfect_Trip


# add null_count

In [12]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

def train_xgboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not XGBOOST_AVAILABLE:
        return None, None, None
    
    print("\n" + "="*80)
    print("Training XGBoost Models")
    print("="*80)
    
    params = config.get_xgboost_params(use_gpu=use_gpu)
    params['num_class'] = len(np.unique(y_train))
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=config.RANDOM_STATE)
    
    oof_predictions = np.zeros((len(X_train), len(np.unique(y_train))))
    test_predictions = np.zeros((len(X_test), len(np.unique(y_train))))
    
    fold_scores = []
    models = []
    
    pbar = tqdm(enumerate(skf.split(X_train, y_train), 1), total=n_folds, desc="XGBoost Folds")
    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"XGBoost Fold {fold}/{n_folds}")
        
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(X_test)
        
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dtrain, 'train'), (dval, 'valid')],
            custom_metric=macro_f1_eval,
            early_stopping_rounds=50,
            verbose_eval=100
        )
        
        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / n_folds
        
        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average='macro')
        fold_scores.append(fold_score)

        print(f"\n{'='*60}")
        print(f"FOLD {fold} SUMMARY:")
        print(f"{'='*60}")
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.best_iteration}")
        print(f"  Best Score (mlogloss): {model.best_score:.6f}")
        
        # Detailed classification report
        oof_fold_pred = np.argmax(oof_predictions[val_idx], axis=1)
        print(f"\n  Per-Class F1 Scores:")
        from sklearn.metrics import classification_report
        print(classification_report(y_val, oof_fold_pred, 
                                   target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
                                   digits=4))
        print(f"{'='*60}\n")
        
        pbar.set_postfix({'F1': f'{fold_score:.6f}'})
        
        models.append(model)
        gc.collect()
    
    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average='macro')
    
        
    print("\n" + "="*80)
    print("XGBOOST TRAINING SUMMARY")
    print("="*80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")
    print(f"\nFold-by-Fold Scores:")
    for i, score in enumerate(fold_scores, 1):
        print(f"  Fold {i}: {score:.6f}")
    
    # Overall classification report pada OOF predictions
    print(f"\n" + "="*80)
    print("OVERALL OUT-OF-FOLD PREDICTIONS REPORT")
    print("="*80)
    print(classification_report(y_train, oof_pred_labels, 
                               target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
                               digits=4))
    print("="*80)

    return test_predictions, models, overall_score


if XGBOOST_AVAILABLE:
    xgboost_test_pred, xgboost_models, xgboost_cv_score = train_xgboost(
        X_train, y_train, X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config['xgboost_gpu']
    )
else:
    xgboost_test_pred, xgboost_models, xgboost_cv_score = None, None, 0.0


Training XGBoost Models
  XGBoost: GPU mode activated


XGBoost Folds:   0%|          | 0/5 [00:00<?, ?it/s]

[0]	train-mlogloss:1.40426	train-macro_f1:0.14189	valid-mlogloss:1.40428	valid-macro_f1:0.14189
[49]	train-mlogloss:0.67870	train-macro_f1:0.57613	valid-mlogloss:0.67905	valid-macro_f1:0.57609

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.576105
  Best Iteration: 0
  Best Score (mlogloss): 0.141886

  Per-Class F1 Scores:
              precision    recall  f1-score   support

     Class_0     0.9966    0.9787    0.9876     80062
     Class_1     1.0000    0.0039    0.0078    160358
     Class_2     0.7258    0.9949    0.8393    879522
     Class_3     0.9963    0.9486    0.9719    320319
     Class_4     0.6212    0.0393    0.0739    159739

    accuracy                         0.7901   1600000
   macro avg     0.8680    0.5931    0.5761   1600000
weighted avg     0.8106    0.7901    0.7135   1600000


[0]	train-mlogloss:1.40427	train-macro_f1:0.14189	valid-mlogloss:1.40427	valid-macro_f1:0.14189
[50]	train-mlogloss:0.67572	train-macro_f1:0.57618	valid-mlogloss:0.67565	valid-macro_f1:0.5

In [12]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False

def train_xgboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not XGBOOST_AVAILABLE:
        return None, None, None
    
    print("\n" + "="*80)
    print("Training XGBoost Models")
    print("="*80)
    
    params = config.get_xgboost_params(use_gpu=use_gpu)
    params['num_class'] = len(np.unique(y_train))
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=config.RANDOM_STATE)
    
    oof_predictions = np.zeros((len(X_train), len(np.unique(y_train))))
    test_predictions = np.zeros((len(X_test), len(np.unique(y_train))))
    
    fold_scores = []
    models = []
    
    pbar = tqdm(enumerate(skf.split(X_train, y_train), 1), total=n_folds, desc="XGBoost Folds")
    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"XGBoost Fold {fold}/{n_folds}")
        
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]
        
        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(X_test)
        
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dtrain, 'train'), (dval, 'valid')],
            custom_metric=macro_f1_eval,
            early_stopping_rounds=50,
            verbose_eval=100
        )
        
        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / n_folds
        
        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average='macro')
        fold_scores.append(fold_score)

        print(f"\n{'='*60}")
        print(f"FOLD {fold} SUMMARY:")
        print(f"{'='*60}")
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.best_iteration}")
        print(f"  Best Score (mlogloss): {model.best_score:.6f}")
        
        # Detailed classification report
        oof_fold_pred = np.argmax(oof_predictions[val_idx], axis=1)
        print(f"\n  Per-Class F1 Scores:")
        from sklearn.metrics import classification_report
        print(classification_report(y_val, oof_fold_pred, 
                                   target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
                                   digits=4))
        print(f"{'='*60}\n")
        
        pbar.set_postfix({'F1': f'{fold_score:.6f}'})
        
        models.append(model)
        gc.collect()
    
    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average='macro')
    
        
    print("\n" + "="*80)
    print("XGBOOST TRAINING SUMMARY")
    print("="*80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")
    print(f"\nFold-by-Fold Scores:")
    for i, score in enumerate(fold_scores, 1):
        print(f"  Fold {i}: {score:.6f}")
    
    # Overall classification report pada OOF predictions
    print(f"\n" + "="*80)
    print("OVERALL OUT-OF-FOLD PREDICTIONS REPORT")
    print("="*80)
    print(classification_report(y_train, oof_pred_labels, 
                               target_names=[f"Class_{i}" for i in range(len(np.unique(y_train)))],
                               digits=4))
    print("="*80)

    return test_predictions, models, overall_score


if XGBOOST_AVAILABLE:
    xgboost_test_pred, xgboost_models, xgboost_cv_score = train_xgboost(
        X_train, y_train, X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config['xgboost_gpu']
    )
else:
    xgboost_test_pred, xgboost_models, xgboost_cv_score = None, None, 0.0


Training XGBoost Models
  XGBoost: GPU mode activated


XGBoost Folds:   0%|          | 0/5 [00:00<?, ?it/s]

[0]	train-mlogloss:1.40426	train-macro_f1:0.14189	valid-mlogloss:1.40428	valid-macro_f1:0.14189
[49]	train-mlogloss:0.67856	train-macro_f1:0.57618	valid-mlogloss:0.67890	valid-macro_f1:0.57614

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.576146
  Best Iteration: 0
  Best Score (mlogloss): 0.141886

  Per-Class F1 Scores:
              precision    recall  f1-score   support

     Class_0     0.9967    0.9787    0.9876     80062
     Class_1     1.0000    0.0039    0.0078    160358
     Class_2     0.7258    0.9950    0.8394    879522
     Class_3     0.9968    0.9486    0.9721    320319
     Class_4     0.6207    0.0393    0.0739    159739

    accuracy                         0.7902   1600000
   macro avg     0.8680    0.5931    0.5761   1600000
weighted avg     0.8106    0.7902    0.7136   1600000


[0]	train-mlogloss:1.40427	train-macro_f1:0.14189	valid-mlogloss:1.40427	valid-macro_f1:0.14189
[50]	train-mlogloss:0.67562	train-macro_f1:0.57620	valid-mlogloss:0.67554	valid-macro_f1:0.5

In [13]:
import optuna
from tqdm import trange

def hyperparameter_tuning(X, y, X_test, n_trials=30):
    def objective(trial):
        w1 = trial.suggest_float("w1", 1.0, 6.0)
        w4 = trial.suggest_float("w4", 1.0, 6.0)

        max_depth = trial.suggest_int("max_depth", 4, 8)
        eta = trial.suggest_float("eta", 0.02, 0.15, log=True)
        subsample = trial.suggest_float("subsample", 0.6, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
        min_child_weight = trial.suggest_float("min_child_weight", 1.0, 10.0)

        sample_weight = np.ones(len(y))
        sample_weight[y == 1] = w1
        sample_weight[y == 4] = w4

        params = config.get_xgboost_params(use_gpu=gpu_config['xgboost_gpu'])
        params.update({
            "max_depth": max_depth,
            "eta": eta,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree,
            "min_child_weight": min_child_weight,
            "num_class": len(np.unique(y))
        })

        _, _, score = train_xgboost(
            X,
            y,
            X_test,
            use_gpu=gpu_config['xgboost_gpu'],
            sample_weight=sample_weight,
            override_params=params
        )

        print(
            "F1={:.6f} | ".format(score) +
            " | ".join([f"{k}={v}" for k, v in trial.params.items()])
        )

        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Best Params:", study.best_params)
    print("Best F1:", study.best_value)

    return study.best_params

In [14]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False


def train_xgboost(
    X_train,
    y_train,
    X_test,
    use_gpu=False,
    sample_weight=None,
    override_params=None
):
    if not XGBOOST_AVAILABLE:
        return None, None, None

    # ===== PARAMS =====
    if override_params is not None:
        params = override_params.copy()
    else:
        params = config.get_xgboost_params(use_gpu=use_gpu)

    params['num_class'] = len(np.unique(y_train))

    # ===== SPLIT =====
    if sample_weight is not None:
        X_tr, X_val, y_tr, y_val, w_tr, w_val = train_test_split(
            X_train,
            y_train,
            sample_weight,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
    else:
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
        w_tr = w_val = None

    # ===== DMATRIX =====
    dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dval = xgb.DMatrix(X_val, label=y_val, weight=w_val)
    dtest = xgb.DMatrix(X_test)

    # ===== TRAIN =====
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dval, 'valid')],
        custom_metric=macro_f1_eval,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # ===== VALID SCORE =====
    val_preds = model.predict(dval)
    val_labels = np.argmax(val_preds, axis=1)
    score = f1_score(y_val, val_labels, average='macro')

    # ===== TEST =====
    test_preds = model.predict(dtest)

    return test_preds, model, score

In [15]:
best_params = hyperparameter_tuning(X_train, y_train, X_test, n_trials=50)

[I 2026-01-13 22:15:20,905] A new study created in memory with name: no-name-15fe8728-2271-4d59-857b-33d598ef369d


  XGBoost: GPU mode activated


[I 2026-01-13 22:16:09,909] Trial 0 finished with value: 0.6310361917783958 and parameters: {'w1': 3.2634612952353637, 'w4': 4.504920105872557, 'max_depth': 4, 'eta': 0.1493074963196003, 'subsample': 0.7299641417876529, 'colsample_bytree': 0.9043154815585304, 'min_child_weight': 9.6461653118132}. Best is trial 0 with value: 0.6310361917783958.


F1=0.631036 | w1=3.2634612952353637 | w4=4.504920105872557 | max_depth=4 | eta=0.1493074963196003 | subsample=0.7299641417876529 | colsample_bytree=0.9043154815585304 | min_child_weight=9.6461653118132
  XGBoost: GPU mode activated


[I 2026-01-13 22:17:02,952] Trial 1 finished with value: 0.6027736028402632 and parameters: {'w1': 1.166506394404956, 'w4': 3.5812803934614603, 'max_depth': 7, 'eta': 0.0920367449628818, 'subsample': 0.6683530682518063, 'colsample_bytree': 0.9989251940166395, 'min_child_weight': 2.2649097720211886}. Best is trial 0 with value: 0.6310361917783958.


...

F1=0.603176 | w1=1.7976865910387194 | w4=2.6284488193451976 | max_depth=6 | eta=0.06686447348107966 | subsample=0.7834397302620884 | colsample_bytree=0.8434209541956927 | min_child_weight=4.827620932660679
  XGBoost: GPU mode activated


[I 2026-01-13 22:57:16,995] Trial 48 finished with value: 0.6303912082904253 and parameters: {'w1': 4.405517165672065, 'w4': 3.7433829274117842, 'max_depth': 5, 'eta': 0.03797687434784198, 'subsample': 0.8803335455858771, 'colsample_bytree': 0.717401583735417, 'min_child_weight': 5.29918251991441}. Best is trial 16 with value: 0.6326100044180505.


F1=0.630391 | w1=4.405517165672065 | w4=3.7433829274117842 | max_depth=5 | eta=0.03797687434784198 | subsample=0.8803335455858771 | colsample_bytree=0.717401583735417 | min_child_weight=5.29918251991441
  XGBoost: GPU mode activated


[I 2026-01-13 22:58:11,274] Trial 49 finished with value: 0.5632284500456928 and parameters: {'w1': 5.791964743784782, 'w4': 4.139639209114184, 'max_depth': 7, 'eta': 0.047200594222630225, 'subsample': 0.6620192774558664, 'colsample_bytree': 0.7798403154807716, 'min_child_weight': 1.711705316836877}. Best is trial 16 with value: 0.6326100044180505.


F1=0.563228 | w1=5.791964743784782 | w4=4.139639209114184 | max_depth=7 | eta=0.047200594222630225 | subsample=0.6620192774558664 | colsample_bytree=0.7798403154807716 | min_child_weight=1.711705316836877
Best Params: {'w1': 5.241971394584772, 'w4': 5.217392774885659, 'max_depth': 5, 'eta': 0.04829634626518371, 'subsample': 0.6583495463269837, 'colsample_bytree': 0.7422343411250392, 'min_child_weight': 4.531142787052159}
Best F1: 0.6326100044180505


In [16]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    XGBOOST_AVAILABLE = False


def train_xgboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not XGBOOST_AVAILABLE:
        return None, None, None, None

    print("\n" + "=" * 80)
    print("Training XGBoost Models")
    print("=" * 80)

    params = config.get_xgboost_params(use_gpu=use_gpu)
    params["num_class"] = len(np.unique(y_train))

    params.update({
        k: v for k, v in best_params.items()
        if k not in ["w1", "w4"]
    })

    sample_weight = np.ones(len(y_train))
    sample_weight[y_train == 1] = best_params["w1"]
    sample_weight[y_train == 4] = best_params["w4"]

    skf = StratifiedKFold(
        n_splits=n_folds,
        shuffle=True,
        random_state=config.RANDOM_STATE
    )

    oof_predictions = np.zeros((len(X_train), params["num_class"]))
    test_predictions = np.zeros((len(X_test), params["num_class"]))

    fold_scores = []
    models = []

    pbar = tqdm(
        enumerate(skf.split(X_train, y_train), 1),
        total=n_folds,
        desc="XGBoost Folds"
    )

    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"XGBoost Fold {fold}/{n_folds}")

        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        dtrain = xgb.DMatrix(
            X_tr,
            label=y_tr,
            weight=sample_weight[train_idx]
        )
        dval = xgb.DMatrix(
            X_val,
            label=y_val,
            weight=sample_weight[val_idx]
        )
        dtest = xgb.DMatrix(X_test)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=1000,
            evals=[(dtrain, "train"), (dval, "valid")],
            custom_metric=macro_f1_eval,
            early_stopping_rounds=50,
            verbose_eval=100
        )

        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / n_folds

        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average="macro")
        fold_scores.append(fold_score)

        print("\n" + "=" * 60)
        print(f"FOLD {fold} SUMMARY:")
        print("=" * 60)
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.best_iteration}")
        print(f"  Best Score (mlogloss): {model.best_score:.6f}")

        print(
            classification_report(
                y_val,
                oof_pred_labels,
                target_names=[
                    f"Class_{i}"
                    for i in range(params["num_class"])
                ],
                digits=4
            )
        )
        print("=" * 60 + "\n")

        pbar.set_postfix({"F1": f"{fold_score:.6f}"})

        models.append(model)
        gc.collect()

    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average="macro")

    print("\n" + "=" * 80)
    print("XGBOOST TRAINING SUMMARY")
    print("=" * 80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")

    return oof_predictions, test_predictions, models, overall_score


if XGBOOST_AVAILABLE:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = train_xgboost(
        X_train,
        y_train,
        X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config["xgboost_gpu"]
    )
else:
    oof_predictions, test_predictions, xgboost_models, xgboost_cv_score = None, None, None, 0.0


Training XGBoost Models
  XGBoost: GPU mode activated


XGBoost Folds:   0%|          | 0/5 [00:00<?, ?it/s]

[0]	train-mlogloss:1.51942	train-macro_f1:0.21046	valid-mlogloss:1.51942	valid-macro_f1:0.21103
[50]	train-mlogloss:1.01971	train-macro_f1:0.63201	valid-mlogloss:1.01974	valid-macro_f1:0.63236

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.632360
  Best Iteration: 0
  Best Score (mlogloss): 0.211031
              precision    recall  f1-score   support

     Class_0     0.9965    0.9780    0.9871     80062
     Class_1     0.2529    0.1378    0.1784    160358
     Class_2     0.7348    0.8163    0.7734    879522
     Class_3     0.9960    0.9474    0.9711    320319
     Class_4     0.2579    0.2458    0.2517    159739

    accuracy                         0.7257   1600000
   macro avg     0.6476    0.6251    0.6324   1600000
weighted avg     0.7043    0.7257    0.7120   1600000


[0]	train-mlogloss:1.51941	train-macro_f1:0.21046	valid-mlogloss:1.51940	valid-macro_f1:0.21036
[50]	train-mlogloss:1.01974	train-macro_f1:0.63186	valid-mlogloss:1.01944	valid-macro_f1:0.63212

FOLD 2 SUMMARY:
  

In [17]:
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80)

if test_predictions is not None:
    print("\n✓ XGBoost Model Successfully Trained")
    print(f"  Cross-Validation Score: {xgboost_cv_score:.6f}")
    print(f"  Model Type: XGBoost with GPU acceleration")

    print("\n" + "="*80)
    print("GENERATING PREDICTIONS ON TEST SET")
    print("="*80)

    final_predictions = test_predictions
    final_pred_labels = np.argmax(final_predictions, axis=1)

    print("\n✓ Predictions Generated Successfully")
    print(f"  Total test samples: {len(final_pred_labels):,}")
    print(f"  Prediction shape: {final_predictions.shape}")
    print(f"  Classes predicted: {len(np.unique(final_pred_labels))}")

    print("\n" + "="*80)
    print("PREDICTION DISTRIBUTION")
    print("="*80)

    unique, counts = np.unique(final_pred_labels, return_counts=True)
    for class_idx, count in zip(unique, counts):
        percentage = (count / len(final_pred_labels)) * 100
        print(f"  Class {class_idx}: {count:,} samples ({percentage:.2f}%)")
else:
    raise ValueError("XGBoost model training failed! Cannot generate predictions.")

print("\n" + "="*80)


MODEL EVALUATION

✓ XGBoost Model Successfully Trained
  Cross-Validation Score: 0.631876
  Model Type: XGBoost with GPU acceleration

GENERATING PREDICTIONS ON TEST SET

✓ Predictions Generated Successfully
  Total test samples: 4,000,000
  Prediction shape: (4000000, 5)
  Classes predicted: 5

PREDICTION DISTRIBUTION
  Class 0: 198,092 samples (4.95%)
  Class 1: 211,865 samples (5.30%)
  Class 2: 2,366,351 samples (59.16%)
  Class 3: 789,866 samples (19.75%)
  Class 4: 433,826 samples (10.85%)



In [18]:
def create_submission(test_ids, predictions, le_target, filename='submission.csv'):
    pred_labels = le_target.inverse_transform(predictions)
    
    submission = pd.DataFrame({
        config.ID_COL: test_ids,
        config.TARGET_COL: pred_labels
    })
    
    submission.to_csv(filename, index=False)
    
    print(f"\nSubmission saved to: {filename}")
    print(f"Submission shape: {submission.shape}")
    print(f"\nPrediction distribution:")
    print(submission[config.TARGET_COL].value_counts())
    
    return submission

test_ids = test[config.ID_COL].values
submission = create_submission(test_ids, final_pred_labels, le_target, 'anava_deira_11.csv')
submission


Submission saved to: anava_deira_11.csv
Submission shape: (4000000, 2)

Prediction distribution:
Trip_Label
Perfect_Trip         2366351
Safety_Violation      789866
Service_Complaint     433826
Navigation_Issue      211865
Fraud_Indication      198092
Name: count, dtype: int64


,Trip_ID,Trip_Label
0,TRIP-06583736,Perfect_Trip
1,TRIP-11356251,Perfect_Trip
2,TRIP-03320505,Service_Complaint
3,TRIP-07188814,Perfect_Trip
4,TRIP-06994869,Perfect_Trip
...,...,...
3999995,TRIP-02234490,Perfect_Trip
3999996,TRIP-04304573,Perfect_Trip
3999997,TRIP-10081352,Perfect_Trip
3999998,TRIP-06550635,Service_Complaint


## catboost

In [13]:
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    print("CatBoost not available. Install with: pip install catboost")
    CATBOOST_AVAILABLE = False


def train_catboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    if not CATBOOST_AVAILABLE:
        return None, None, 0.0

    print("\n" + "="*80)
    print("Training CatBoost Models")
    print("="*80)

    params = config.get_catboost_params(use_gpu=use_gpu)
    num_class = len(np.unique(y_train))

    skf = StratifiedKFold(
        n_splits=n_folds,
        shuffle=True,
        random_state=config.RANDOM_STATE
    )

    oof_predictions = np.zeros((len(X_train), num_class))
    test_predictions = np.zeros((len(X_test), num_class))

    fold_scores = []
    models = []

    pbar = tqdm(
        enumerate(skf.split(X_train, y_train), 1),
        total=n_folds,
        desc="CatBoost Folds"
    )

    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"CatBoost Fold {fold}/{n_folds}")

        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model = CatBoostClassifier(**params)

        model.fit(
            X_tr,
            y_tr,
            eval_set=(X_val, y_val),
            use_best_model=True
        )

        oof_predictions[val_idx] = model.predict_proba(X_val)
        test_predictions += model.predict_proba(X_test) / n_folds

        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average='macro')
        fold_scores.append(fold_score)

        print(f"\n{'='*60}")
        print(f"FOLD {fold} SUMMARY:")
        print(f"{'='*60}")
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.get_best_iteration()}")

        oof_fold_pred = np.argmax(oof_predictions[val_idx], axis=1)
        print(f"\n  Per-Class F1 Scores:")
        print(classification_report(
            y_val,
            oof_fold_pred,
            target_names=[f"Class_{i}" for i in range(num_class)],
            digits=4
        ))
        print(f"{'='*60}\n")

        pbar.set_postfix({'F1': f'{fold_score:.6f}'})
        models.append(model)
        gc.collect()

    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average='macro')

    print("\n" + "="*80)
    print("CATBOOST TRAINING SUMMARY")
    print("="*80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")
    print("\nFold-by-Fold Scores:")
    for i, score in enumerate(fold_scores, 1):
        print(f"  Fold {i}: {score:.6f}")

    print("\n" + "="*80)
    print("OVERALL OUT-OF-FOLD PREDICTIONS REPORT")
    print("="*80)
    print(classification_report(
        y_train,
        oof_pred_labels,
        target_names=[f"Class_{i}" for i in range(num_class)],
        digits=4
    ))
    print("="*80)

    return test_predictions, models, overall_score


if CATBOOST_AVAILABLE:
    cb_test_pred, cb_models, cb_cv_score = train_catboost(
        X_train,
        y_train,
        X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config['catboost_gpu']
    )
else:
    cb_test_pred, cb_models, cb_cv_score = None, None, 0.0


Training CatBoost Models
  CatBoost: GPU mode activated


CatBoost Folds:   0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.5353753	test: 0.5356663	best: 0.5356663 (0)	total: 397ms	remaining: 6m 36s
100:	learn: 0.6336824	test: 0.6332246	best: 0.6332246 (100)	total: 11.7s	remaining: 1m 44s
bestTest = 0.6349589593
bestIteration = 118
Shrink model to first 119 iterations.

FOLD 1 SUMMARY:
  Validation F1 (Macro): 0.582352
  Best Iteration: 118

  Per-Class F1 Scores:
              precision    recall  f1-score   support

     Class_0     0.9315    0.9821    0.9561     80062
     Class_1     0.1707    0.3783    0.2353    160358
     Class_2     0.7457    0.3563    0.4822    879522
     Class_3     0.9840    0.9517    0.9676    320319
     Class_4     0.1855    0.4995    0.2705    159739

    accuracy                         0.5233   1600000
   macro avg     0.6035    0.6336    0.5824   1600000
weighted avg     0.6892    0.5233    0.5572   1600000


0:	learn: 0.5300420	test: 0.5299114	best: 0.5299114 (0)	total: 148ms	remaining: 2m 28s
100:	learn: 0.6314820	test: 0.6316985	best: 0.6316985 (100)	total:

In [23]:
import optuna
from tqdm import trange

def hyperparameter_tuning(X, y, X_test, n_trials=30):
    def objective(trial):
        w1 = trial.suggest_float("w1", 1.0, 6.0)
        w4 = trial.suggest_float("w4", 1.0, 6.0)

        depth = trial.suggest_int("depth", 5, 9)
        learning_rate = trial.suggest_float("learning_rate", 0.03, 0.15, log=True)
        l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 2.0, 10.0)
        bagging_temperature = trial.suggest_float("bagging_temperature", 0.0, 1.0)

        sample_weight = np.ones(len(y))
        sample_weight[y == 1] = w1
        sample_weight[y == 4] = w4

        params = config.get_catboost_params(use_gpu=gpu_config['catboost_gpu'])
        params.update({
            "depth": depth,
            "learning_rate": learning_rate,
            "l2_leaf_reg": l2_leaf_reg,
            "bagging_temperature": bagging_temperature
        })

        _, _, score = train_catboost(
            X,
            y,
            X_test,
            use_gpu=gpu_config['catboost_gpu'],
            sample_weight=sample_weight,
            override_params=params
        )

        print(
            "F1={:.6f} | ".format(score) +
            " | ".join([f"{k}={v}" for k, v in trial.params.items()])
        )

        return score

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    print("Best Params:", study.best_params)
    print("Best F1:", study.best_value)

    return study.best_params


In [24]:
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    print("CatBoost not available. Install with: pip install catboost")
    CATBOOST_AVAILABLE = False


def train_catboost(
    X_train,
    y_train,
    X_test,
    use_gpu=False,
    sample_weight=None,
    override_params=None
):
    if not CATBOOST_AVAILABLE:
        return None, None, None

    if override_params is not None:
        params = override_params.copy()
    else:
        params = config.get_catboost_params(use_gpu=use_gpu)

    if sample_weight is not None:
        X_tr, X_val, y_tr, y_val, w_tr, w_val = train_test_split(
            X_train,
            y_train,
            sample_weight,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
    else:
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            stratify=y_train,
            random_state=config.RANDOM_STATE
        )
        w_tr = w_val = None

    model = CatBoostClassifier(**params)

    model.fit(
        X_tr,
        y_tr,
        sample_weight=w_tr,
        eval_set=(X_val, y_val),
        use_best_model=True
    )

    val_preds = model.predict_proba(X_val)
    val_labels = np.argmax(val_preds, axis=1)
    score = f1_score(y_val, val_labels, average='macro')

    test_preds = model.predict_proba(X_test)

    return test_preds, model, score


In [25]:
best_params = hyperparameter_tuning(X_train, y_train, X_test, n_trials=30)

[I 2026-01-14 09:46:11,717] A new study created in memory with name: no-name-366ff937-d1fb-47fa-83eb-adf4eba1dca4


  CatBoost: GPU mode activated
0:	learn: 0.5863739	test: 0.6209939	best: 0.6209939 (0)	total: 213ms	remaining: 3m 32s
bestTest = 0.6286664048
bestIteration = 46
Shrink model to first 47 iterations.


[I 2026-01-14 09:46:45,618] Trial 0 finished with value: 0.6205408097407379 and parameters: {'w1': 4.210444733453039, 'w4': 3.2328810908559182, 'depth': 8, 'learning_rate': 0.04675687260969071, 'l2_leaf_reg': 4.971140172994592, 'bagging_temperature': 0.30986099765143993}. Best is trial 0 with value: 0.6205408097407379.


F1=0.620541 | w1=4.210444733453039 | w4=3.2328810908559182 | depth=8 | learning_rate=0.04675687260969071 | l2_leaf_reg=4.971140172994592 | bagging_temperature=0.30986099765143993
  CatBoost: GPU mode activated
0:	learn: 0.5902393	test: 0.6022778	best: 0.6022778 (0)	total: 197ms	remaining: 3m 16s
bestTest = 0.6328935357
bestIteration = 28
Shrink model to first 29 iterations.


[I 2026-01-14 09:47:16,659] Trial 1 finished with value: 0.6072397562194156 and parameters: {'w1': 3.2709275408378122, 'w4': 1.0636383004237544, 'depth': 8, 'learning_rate': 0.0943727061496995, 'l2_leaf_reg': 2.2040992786698643, 'bagging_temperature': 0.5298138141788649}. Best is trial 0 with value: 0.6205408097407379.


...

F1=0.622400 | w1=4.176391848776233 | w4=4.182540989802282 | depth=8 | learning_rate=0.13047712112340862 | l2_leaf_reg=2.78538489247834 | bagging_temperature=0.43079441857029555
  CatBoost: GPU mode activated
0:	learn: 0.5458196	test: 0.5081040	best: 0.5081040 (0)	total: 168ms	remaining: 2m 47s
bestTest = 0.6244724829
bestIteration = 21
Shrink model to first 22 iterations.


[I 2026-01-14 10:01:22,890] Trial 28 finished with value: 0.6222216755378245 and parameters: {'w1': 2.912487488937776, 'w4': 4.745486162831782, 'depth': 7, 'learning_rate': 0.10634934547388318, 'l2_leaf_reg': 4.105633445902795, 'bagging_temperature': 0.6034017639061069}. Best is trial 26 with value: 0.6226653916052056.


F1=0.622222 | w1=2.912487488937776 | w4=4.745486162831782 | depth=7 | learning_rate=0.10634934547388318 | l2_leaf_reg=4.105633445902795 | bagging_temperature=0.6034017639061069
  CatBoost: GPU mode activated
0:	learn: 0.5902393	test: 0.6254316	best: 0.6254316 (0)	total: 202ms	remaining: 3m 21s
bestTest = 0.6268661218
bestIteration = 12
Shrink model to first 13 iterations.


[I 2026-01-14 10:01:51,429] Trial 29 finished with value: 0.6226753377517147 and parameters: {'w1': 4.509194597384958, 'w4': 4.96455487505969, 'depth': 8, 'learning_rate': 0.12417373275177125, 'l2_leaf_reg': 5.779911380449872, 'bagging_temperature': 0.37002965644391456}. Best is trial 29 with value: 0.6226753377517147.


F1=0.622675 | w1=4.509194597384958 | w4=4.96455487505969 | depth=8 | learning_rate=0.12417373275177125 | l2_leaf_reg=5.779911380449872 | bagging_temperature=0.37002965644391456
Best Params: {'w1': 4.509194597384958, 'w4': 4.96455487505969, 'depth': 8, 'learning_rate': 0.12417373275177125, 'l2_leaf_reg': 5.779911380449872, 'bagging_temperature': 0.37002965644391456}
Best F1: 0.6226753377517147


In [29]:
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    print("CatBoost not available. Install with: pip install catboost")
    CATBOOST_AVAILABLE = False


def train_catboost(X_train, y_train, X_test, n_folds=5, use_gpu=False):
    print("\n" + "=" * 80)
    print("Training CatBoost Models")
    print("=" * 80)

    num_classes = len(np.unique(y_train))

    # === CLASS WEIGHT (ikut logic w1 & w4 kamu) ===
    class_weights = np.ones(num_classes)
    class_weights[1] = best_params["w1"]
    class_weights[4] = best_params["w4"]

    # === PARAMS ===
    params = {
        "loss_function": "MultiClass",
        "eval_metric": "TotalF1",
        "iterations": 3000,
        "learning_rate": best_params["learning_rate"],
        "depth": best_params["depth"],
        "l2_leaf_reg": best_params["l2_leaf_reg"],
        "bagging_temperature": best_params["bagging_temperature"],
        "class_weights": class_weights.tolist(),
        "random_seed": config.RANDOM_STATE,
        "early_stopping_rounds": 100,
        "verbose": 100,
        "task_type": "GPU" if use_gpu else "CPU",
    }

    skf = StratifiedKFold(
        n_splits=n_folds,
        shuffle=True,
        random_state=config.RANDOM_STATE
    )

    oof_predictions = np.zeros((len(X_train), num_classes))
    test_predictions = np.zeros((len(X_test), num_classes))

    fold_scores = []
    models = []

    pbar = tqdm(
        enumerate(skf.split(X_train, y_train), 1),
        total=n_folds,
        desc="CatBoost Folds"
    )

    for fold, (train_idx, val_idx) in pbar:
        pbar.set_description(f"CatBoost Fold {fold}/{n_folds}")

        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        model = CatBoostClassifier(**params)

        model.fit(
            X_tr,
            y_tr,
            eval_set=(X_val, y_val),
            use_best_model=True
        )

        oof_predictions[val_idx] = model.predict_proba(X_val)
        test_predictions += model.predict_proba(X_test) / n_folds

        oof_pred_labels = np.argmax(oof_predictions[val_idx], axis=1)
        fold_score = f1_score(y_val, oof_pred_labels, average="macro")
        fold_scores.append(fold_score)

        print("\n" + "=" * 60)
        print(f"FOLD {fold} SUMMARY:")
        print("=" * 60)
        print(f"  Validation F1 (Macro): {fold_score:.6f}")
        print(f"  Best Iteration: {model.get_best_iteration()}")

        print(
            classification_report(
                y_val,
                oof_pred_labels,
                target_names=[f"Class_{i}" for i in range(num_classes)],
                digits=4
            )
        )
        print("=" * 60 + "\n")

        pbar.set_postfix({"F1": f"{fold_score:.6f}"})

        models.append(model)
        gc.collect()

    oof_pred_labels = np.argmax(oof_predictions, axis=1)
    overall_score = f1_score(y_train, oof_pred_labels, average="macro")

    print("\n" + "=" * 80)
    print("CATBOOST TRAINING SUMMARY")
    print("=" * 80)
    print(f"Overall CV Score (Macro F1): {overall_score:.6f}")
    print(f"Standard Deviation: {np.std(fold_scores):.6f}")
    print(f"Min F1 Score: {np.min(fold_scores):.6f}")
    print(f"Max F1 Score: {np.max(fold_scores):.6f}")

    return oof_predictions, test_predictions, models, overall_score

if CATBOOST_AVAILABLE:
    cb_oof_pred, cb_test_pred, cb_models, cb_cv_score = train_catboost(
        X_train,
        y_train,
        X_test,
        n_folds=config.N_FOLDS,
        use_gpu=gpu_config['catboost_gpu']
    )

else:
    cb_oof_pred, cb_test_pred, cb_models, cb_cv_score = None, None, 0.0


Training CatBoost Models


CatBoost Folds:   0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.4272947	test: 0.4281549	best: 0.4281549 (0)	total: 110ms	remaining: 5m 30s
100:	learn: 0.4699608	test: 0.4697722	best: 0.4697732 (99)	total: 7.89s	remaining: 3m 46s
200:	learn: 0.4741527	test: 0.4717562	best: 0.4717562 (200)	total: 16.4s	remaining: 3m 47s
300:	learn: 0.4785393	test: 0.4722303	best: 0.4723355 (294)	total: 25.6s	remaining: 3m 49s
400:	learn: 0.4830399	test: 0.4728535	best: 0.4729747 (392)	total: 35s	remaining: 3m 46s
500:	learn: 0.4872333	test: 0.4731926	best: 0.4732673 (486)	total: 44.2s	remaining: 3m 40s
600:	learn: 0.4919702	test: 0.4736170	best: 0.4737791 (589)	total: 53.4s	remaining: 3m 33s
700:	learn: 0.4969242	test: 0.4740465	best: 0.4741065 (695)	total: 1m 2s	remaining: 3m 25s
800:	learn: 0.5017972	test: 0.4748827	best: 0.4748827 (800)	total: 1m 11s	remaining: 3m 17s
900:	learn: 0.5062615	test: 0.4748411	best: 0.4750257 (840)	total: 1m 21s	remaining: 3m 9s
1000:	learn: 0.5107265	test: 0.4750901	best: 0.4751585 (991)	total: 1m 30s	remaining: 3m 1s
1100

In [30]:
print("\n" + "="*80)
print("MODEL EVALUATION")
print("="*80)

if cb_test_pred is not None:
    print(f"\n✓ CatBoost Model Successfully Trained")
    print(f"  Cross-Validation Score: {cb_cv_score:.6f}")
    print(f"  Model Type: CatBoost with {'GPU' if gpu_config['catboost_gpu'] else 'CPU'} acceleration")

    print("\n" + "="*80)
    print("GENERATING PREDICTIONS ON TEST SET")
    print("="*80)

    final_predictions = cb_test_pred
    final_pred_labels = np.argmax(final_predictions, axis=1)

    print(f"\n✓ Predictions Generated Successfully")
    print(f"  Total test samples: {len(final_pred_labels):,}")
    print(f"  Prediction shape: {final_predictions.shape}")
    print(f"  Classes predicted: {len(np.unique(final_pred_labels))}")

    print("\n" + "="*80)
    print("PREDICTION DISTRIBUTION")
    print("="*80)

    unique, counts = np.unique(final_pred_labels, return_counts=True)
    for class_idx, count in zip(unique, counts):
        percentage = (count / len(final_pred_labels)) * 100
        print(f"  Class {class_idx}: {count:,} samples ({percentage:.2f}%)")

else:
    raise ValueError("CatBoost model training failed! Cannot generate predictions.")

print("\n" + "="*80)


MODEL EVALUATION

✓ CatBoost Model Successfully Trained
  Cross-Validation Score: 0.624024
  Model Type: CatBoost with GPU acceleration

GENERATING PREDICTIONS ON TEST SET

✓ Predictions Generated Successfully
  Total test samples: 4,000,000
  Prediction shape: (4000000, 5)
  Classes predicted: 5

PREDICTION DISTRIBUTION
  Class 0: 198,190 samples (4.95%)
  Class 1: 228,760 samples (5.72%)
  Class 2: 2,048,676 samples (51.22%)
  Class 3: 791,126 samples (19.78%)
  Class 4: 733,248 samples (18.33%)



In [31]:
def create_submission(test_ids, predictions, le_target, filename='submission.csv'):
    pred_labels = le_target.inverse_transform(predictions)
    
    submission = pd.DataFrame({
        config.ID_COL: test_ids,
        config.TARGET_COL: pred_labels
    })
    
    submission.to_csv(filename, index=False)
    
    print(f"\nSubmission saved to: {filename}")
    print(f"Submission shape: {submission.shape}")
    print(f"\nPrediction distribution:")
    print(submission[config.TARGET_COL].value_counts())
    
    return submission

test_ids = test[config.ID_COL].values
submission = create_submission(test_ids, final_pred_labels, le_target, 'anava_deira_12.csv')
submission


Submission saved to: anava_deira_12.csv
Submission shape: (4000000, 2)

Prediction distribution:
Trip_Label
Perfect_Trip         2048676
Safety_Violation      791126
Service_Complaint     733248
Navigation_Issue      228760
Fraud_Indication      198190
Name: count, dtype: int64


,Trip_ID,Trip_Label
0,TRIP-06583736,Perfect_Trip
1,TRIP-11356251,Service_Complaint
2,TRIP-03320505,Service_Complaint
3,TRIP-07188814,Perfect_Trip
4,TRIP-06994869,Perfect_Trip
...,...,...
3999995,TRIP-02234490,Perfect_Trip
3999996,TRIP-04304573,Perfect_Trip
3999997,TRIP-10081352,Service_Complaint
3999998,TRIP-06550635,Service_Complaint
